# Prompt Engineering: LLM Practice Seminar

_Daniil Sherki_

This seminar **requires an authorization key** for interaction. If an authorization key is not available, it is possible to sign up via the [GigaChat Studio](https://developers.sber.ru/docs/ru/studio/overview).

## How to get started

1. Clone this repository to the local machine.

2. Install the required dependencies by running the following command:
 

In [1]:
!pip install gigachat python-dotenv

3. Set up the authorization key and model name. If a `.env` is used, set `API_KEY` there (and optionally `MODEL_NAME`). Otherwise replace `"your_gigachat_credentials_here"` with the actual GigaChat authorization key.

In [1]:
import os
from dotenv import load_dotenv
import re

load_dotenv()

GIGACHAT_CREDENTIALS = os.getenv("API_KEY") or os.getenv("GIGACHAT_CREDENTIALS") or ""
MODEL_NAME = os.getenv("MODEL_NAME") or "GigaChat"

# Stores the GIGACHAT_CREDENTIALS & MODEL_NAME variables for use across notebooks within the IPython store
%store GIGACHAT_CREDENTIALS
%store MODEL_NAME


Stored 'GIGACHAT_CREDENTIALS' (str)
Stored 'MODEL_NAME' (str)


4. The notebook cells are executed in order.

---

## Usage Notes & Tips 💡

- This course uses GigaChat with temperature 0. This seminar will talk more about temperature later in the course. For now, it's enough to understand that these settings yield more deterministic results. All prompt engineering techniques in this course also apply to other GigaChat model variants such as GigaChat-Pro and GigaChat-Max.

- `Shift + Enter` executes the current cell and moves to the next one.

- At the bottom of a tutorial page, navigate to the next numbered file in the folder, or to the next numbered folder after the chapter content is complete.

### The GigaChat SDK & the chat/completions API
This seminar will be using the [GigaChat python SDK](https://developers.sber.ru/docs/ru/gigachat/guides/using-sdks) and the [chat/completions API](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-chat) throughout this tutorial. 

Below is an example of what running a prompt will look like in this tutorial. First, we create `get_completion`, which is a helper function that sends a prompt to GigaChat and returns GigaChat's generated response. Run that cell now.

In [20]:
import os
from dotenv import load_dotenv
import re

load_dotenv()

from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole

# Retrieve the GIGACHAT_CREDENTIALS & MODEL_NAME variables from the IPython store
%store -r GIGACHAT_CREDENTIALS
%store -r MODEL_NAME

# Override from .env if present
if os.getenv("API_KEY"):
    GIGACHAT_CREDENTIALS = os.getenv("API_KEY")
elif os.getenv("GIGACHAT_CREDENTIALS"):
    GIGACHAT_CREDENTIALS = os.getenv("GIGACHAT_CREDENTIALS")
if os.getenv("MODEL_NAME"):
    MODEL_NAME = os.getenv("MODEL_NAME")


client = GigaChat(credentials=GIGACHAT_CREDENTIALS, verify_ssl_certs=False)

def _apply_stop_sequences(text, stop_sequences):
    if not stop_sequences:
        return text
    stop_at = None
    for seq in stop_sequences:
        idx = text.find(seq)
        if idx != -1 and (stop_at is None or idx < stop_at):
            stop_at = idx
    return text if stop_at is None else text[:stop_at]


def _normalize_messages(messages, system_prompt="", prefill=""):
    normalized = []
    if system_prompt:
        normalized.append(Messages(role=MessagesRole.SYSTEM, content=system_prompt))
    for message in messages:
        if isinstance(message, Messages):
            normalized.append(message)
            continue
        if isinstance(message, dict):
            role = message.get("role", "user")
            content = message.get("content", "")
        else:
            role = "user"
            content = str(message)
        if role == "assistant":
            role_enum = MessagesRole.ASSISTANT
        elif role == "system":
            role_enum = MessagesRole.SYSTEM
        else:
            role_enum = MessagesRole.USER
        normalized.append(Messages(role=role_enum, content=content))
    if prefill:
        normalized.append(Messages(role=MessagesRole.ASSISTANT, content=prefill))
    return normalized


def get_completion(prompt_or_messages, system_prompt="", prefill="", stop_sequences=None):
    if isinstance(prompt_or_messages, (list, tuple)):
        messages = _normalize_messages(prompt_or_messages, system_prompt=system_prompt, prefill=prefill)
    else:
        messages = []
        if system_prompt:
            messages.append(Messages(role=MessagesRole.SYSTEM, content=system_prompt))
        messages.append(Messages(role=MessagesRole.USER, content=str(prompt_or_messages)))
        if prefill:
            messages.append(Messages(role=MessagesRole.ASSISTANT, content=prefill))
    chat = Chat(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    response = client.chat(chat)
    text = response.choices[0].message.content
    return _apply_stop_sequences(text, stop_sequences)


This section provides an example prompt for GigaChat and prints the output via the `get_completion` helper function. The cell below prints a response from GigaChat.

Optionally, modify the prompt string to observe different responses from GigaChat.

In [14]:
# Prompt
prompt = "Hello, GigaChat!"

# Get GigaChat's response
print(get_completion(prompt))

Привет! Как настроение? Чем займёмся?


The `GIGACHAT_CREDENTIALS` and `MODEL_NAME` variables defined earlier will be used throughout the tutorial. Just make sure to run the cells for each tutorial page from top to bottom.

---

## Part 1

GigaChat provides a [chat/completions API](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-chat) for prompt-based generation. For this tutorial, this seminar will be exclusively using the chat/completions API.

At minimum, a call to GigaChat using the chat/completions API requires the following parameters:
- `model`: the [API model name](https://developers.sber.ru/docs/ru/gigachat/models) intended for the call.

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that GigaChat may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause GigaChat to stop generating mid-word or mid-sentence.

- `messages`: an array of input messages. Models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, prior turns are specified in the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. it is possible to specify a single `user`-role message, or it is possible to include multiple `user` and `assistant` messages (they must alternate, if so). The first message must always use the user `role`.

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in GigaChat's response. For these lessons and exercises, we have set `temperature` to 0.

For a complete list of all API parameters, visit our [API documentation](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-chat).

### Examples

This section reviews how GigaChat responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and GigaChat's response will appear below the block.

In [15]:
# Prompt
PROMPT = "Hi GigaChat, how are you?"

# Print GigaChat's response
print(get_completion(PROMPT))

Привет! У меня всё отлично — готов общаться, помогать разобраться в сложных вопросах или просто поговорить обо всём на свете. Как твои дела?


In [16]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print GigaChat's response
print(get_completion(PROMPT))

Цвет океана варьируется в зависимости от множества факторов: глубины, освещенности, наличия водорослей и планктона, температуры воды, содержания солей и минералов, а также географического положения.

- **На мелководье** (например, вблизи берегов): вода часто имеет зеленоватый оттенок благодаря большому количеству микроскопических растений — фитопланктона.
- **Глубоководные зоны**: цвет становится синим или голубым, поскольку вода хорошо пропускает синий спектр солнечного света, поглощая другие цвета.
- **При наличии большого количества органических веществ или взвеси частиц**: вода приобретает бурый или коричневый оттенок.
- **Во время цветения водорослей**: вода может стать ярко-зелёной или даже красноватой.

Таким образом, океан бывает разных цветов — от глубокого синего до зелёного, бурого и даже красного. В среднем же, типичный цвет открытого океана — светло-голубой или тёмно-синий.


In [17]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print GigaChat's response
print(get_completion(PROMPT))

Céline Marie Claudette Dion was born on **March 30, 1968**.


This section reviews prompts that do not follow correct chat/completions API formatting; such inputs return an error.

First, a chat/completions API call is shown without `role` and `content` fields in the `messages` array.

In [18]:
# Get GigaChat's response
chat = Chat(
    model=MODEL_NAME,
    max_tokens=2000,
    temperature=0.0,
    messages=[
        Messages(role=MessagesRole.USER, content="Hi GigaChat, how are you?")
    ]
)
response = client.chat(chat)

# Print GigaChat's response
print(response.choices[0].message.content)


Привет! У меня всё отлично — бодро мыслю, готов общаться и помогать тебе с разными вопросами. Как твои дела?


Next, a prompt is shown that fails to alternate between the `user` and `assistant` roles.

In [19]:
# Get GigaChat's response
try:
    chat = Chat(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          Messages(role=MessagesRole.USER, content="What year was Celine Dion born in?"),
          Messages(role=MessagesRole.USER, content="Also, can you tell me some other facts about her?")
        ]
    )
    response = client.chat(chat)
    # Print GigaChat's response
    print(response.choices[0].message.content)
except Exception as e:
    print(type(e).__name__ + ":", e)


Céline Marie Claudette Dion was born on **March 30, 1968** in Charlemagne, Quebec, Canada.

### Some interesting facts about Céline Dion:

- **Early Life**: She began singing at a very young age and performed with her family's band, Les Belles Harmonies, from the age of 11.
  
- **Career Breakthrough**: Her breakthrough came when she won the 1982 Yule Show competition in Quebec, which led to her first major recording contract.

- **International Success**: In 1988, she released her debut album *Unison* in French, followed by *Dion*, which became an international hit. Her English-language debut album *Let's Talk About Love* (1993) brought her global fame.

- **Awards & Honors**: Dion has sold over 250 million records worldwide, making her one of the best-selling artists of all time. She is also known for her powerful voice and emotional performances.

- **Personal Life**: She married René Angélil in 1994, who managed her career until his death in 2016. They had two sons together: Rene C

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. Multiple `user` & `assistant` pairs are allowed (simulating a multi-turn conversation). It is also possible to put words into a terminal `assistant` message for GigaChat to continue from that point (more on that in later chapters).

#### System Prompts

System prompts can also be used. A system prompt provides context, instructions, and guidelines to GigaChat before the "User" turn. 

Structurally, system prompts are separate from the `user` & `assistant` messages and belong in a separate `system` parameter (see the `get_completion` helper function in the [Setup](#setup) section). 

Within this material, the completions function includes a `system` field where system prompts are used. If no system prompt is required, set `SYSTEM_PROMPT` to an empty string.

#### System Prompt Example

In [20]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print GigaChat's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

1. What role does sunlight play in determining the color of the sky?
2. How do different wavelengths of light interact with Earth's atmosphere?
3. Why are shorter wavelengths more likely to scatter than longer ones?
4. Can you explain Rayleigh scattering and its significance in this context?
5. Are there any other factors besides atmospheric scattering that contribute to the sky’s appearance?


---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get GigaChat to **count to three.** The output will also indicate whether the solution is correct.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print GigaChat's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

1‒2‒3.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ For a hint, run the cell below!

In [34]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

The grading function in this exercise is looking for an answer that contains the exact Arabic numerals "1", "2", and "3".
You can often get Claude to do what you want simply by asking.


### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make GigaChat respond like it's a 3 year old child.

In [21]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"
# Prompt
PROMPT = "How big is the sky?"

# Get GigaChat's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print GigaChat's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

The "size" of the sky can be understood in several ways:

1. **Visual Perspective**: From a human perspective, the sky appears to stretch from horizon to horizon and seems infinite because we can't see its boundaries.
2. **Geographical Scale**: The sky covers the entire Earth's atmosphere, which extends about 60 miles (100 kilometers) above sea level. This means that the sky encompasses all the air around us and beyond it into space.
3. **Cosmic Scale**: Astronomically speaking, the sky includes everything visible in the nighttime or daytime sky: stars, planets, galaxies, nebulae, etc., extending billions of light-years across the observable universe.

In summary:
- Visually, the sky looks vast but has no physical boundary.
- Geographically, it's the atmosphere surrounding Earth.
- Cosmologically, it's an immense expanse containing countless celestial bodies.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: False


❓ For a hint, run the cell below!

In [16]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

The grading function in this exercise is looking for answers that contain "soo" or "giggles".
There are many ways to solve this, just by asking!


---

## Part 2

**GigaChat responds best to clear and direct instructions.**

GigaChat has no context beyond the prompt. As with instructing a human for the first time, clearer and more specific instructions generally improve response accuracy.
				
When in doubt, follow the **Golden Rule of Clear Prompting**:
- The prompt can be shown to a colleague or friend to validate whether it produces the intended result; if they are confused, the prompt is unclear.

### Examples

This section demonstrates take a task like writing poetry. (Ignore any syllable mismatch - LLMs aren't great at counting syllables yet.)

In [39]:
# Prompt
PROMPT = "Write a haiku about robots."

# Print GigaChat's response
print(get_completion(PROMPT))

## Чугунный мир вокруг  
Роботы ходят толпою —  
Сердца давно в плену

Технологии шумят,  
Но чувства лишь человек  
Дарит живым сердцам


This haiku is nice enough, but users may want GigaChat to go directly into the poem without the "Here is a haiku" preamble.

How do we achieve that? We **ask for it**!

In [40]:
# Prompt
PROMPT = "Write a haiku about robots. Skip the preamble; go straight into the poem."

# Print GigaChat's response
print(get_completion(PROMPT))

Сердца стальные светят,  
Чужды чувства — лишь расчет.  
Роботы — новый век.


This section presents another example. This section demonstrates ask GigaChat who's the best basketball player of all time. it is possible to see below that while GigaChat lists a few names, **it doesn't respond with a definitive "best"**.

In [41]:
# Prompt
PROMPT = "Who is the best basketball player of all time?"

# Print GigaChat's response
print(get_completion(PROMPT))

Выбор лучшего баскетболиста всех времен — субъективная тема, зависящая от множества факторов: стиля игры, достижений, влияния на команду и игру в целом, а также личных предпочтений болельщиков и экспертов.

Однако общепринятые лидеры среди лучших игроков включают:

1. **Майкл Джордан (Michael Jordan)**  
   Американский игрок, один из величайших баскетболистов в истории НБА. Его достижения включают шесть титулов чемпиона НБА, пять MVP регулярного чемпионата, десять титулов самого ценного игрока плей-офф (MVP Finals), десять титулов MVP регулярного сезона и девять раз входил в первую сборную всех звезд лиги.

2. **Леброн Джеймс (LeBron James)**  
   Современный лидер НБА, который уже давно считается одним из лучших игроков своего поколения. Леброн имеет четыре титула чемпиона НБА, три MVP регулярного чемпионата, четыре MVP плей-офф и является многократным лидером всех звезд.

3. **Коби Брайант (Kobe Bryant)**  
   Один из самых успешных игроков в истории НБА, обладатель пяти чемпионских

Can we get GigaChat to make up its mind and decide on a best player? Yes! Just ask!

In [42]:
# Prompt
PROMPT = "Who is the best basketball player of all time? Yes, there are differing opinions, but if you absolutely had to pick one player, who would it be?"

# Print GigaChat's response
print(get_completion(PROMPT))

The question of “who is the best basketball player of all time” often sparks intense debates among fans and experts alike. There’s no definitive answer that everyone agrees on, as different people may prioritize various aspects such as individual statistics, team success, longevity, impact on the game, or cultural influence.

However, if we were to choose a single player based on an objective analysis of their career achievements, here are some key arguments for why **Michael Jordan** (often referred to simply as “Air Jordan”) might be considered the greatest of all time:

### Arguments in favor of Michael Jordan:
1. **Individual Achievements:**
   - Six NBA championships with three MVP awards during his first tenure.
   - Five more titles after returning from retirement.
   - Scored over 32,000 points in his career, averaging nearly 30 points per game.
   - Led the league in scoring five times.
   - Won multiple All-Star MVPs and Finals MVPs.

2. **Consistency and Versatility:**
   - 

---

## Exercises
- [Exercise 2.1 - Spanish](#exercise-21---spanish)
- [Exercise 2.2 - One Player Only](#exercise-22---one-player-only)
- [Exercise 2.3 - Write a Story](#exercise-23---write-a-story)

### Exercise 2.1 - Spanish
Modify the `SYSTEM_PROMPT` to make GigaChat output its answer in Spanish.

In [22]:
# System prompt - this is the only field you should chnage
SYSTEM_PROMPT = "[Replace this text]"
# Prompt
PROMPT = "Hello GigaChat, how are you?"

# Get GigaChat's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return "hola" in text.lower()

# Print GigaChat's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

I'm doing great, thanks for asking! How can I assist you today?

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: False


❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_2_1_hint; print(exercise_2_1_hint)

### Exercise 2.2 - One Player Only

Modify the `PROMPT` so that GigaChat doesn't equivocate at all and responds with **ONLY** the name of one specific player, with **no other words or punctuation**. 

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return text == "Michael Jordan"

# Print GigaChat's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

LeBron James

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: False


❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_2_2_hint; print(exercise_2_2_hint)

### Exercise 2.3 - Write a Story

Modify the `PROMPT` so that GigaChat responds with as long a response as it is possible to muster. Answers over **800 words** are graded as correct.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    trimmed = text.strip()
    words = len(trimmed.split())
    return words >= 800

# Print GigaChat's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

# **Гигантский эпический стих**

## Часть первая: Пробуждение

### Утро великое встаёт,
Солнце над миром восходит вновь.
Звёзды погасли, ночь отлетела прочь,
И мир пробуждается вновь живой.

### Ветер шепчет древним сказаньям,
Шумит листва вековых дубрав.
Птицы летят сквозь облака,
Зовут рассвета светлый час.

### Река течёт, журчит вода,
Тихо струится меж камней.
Луг зелёный травою одет,
Свет солнца ласково льётся кругом.

---

## Часть вторая: Путешествие героя

### Герой наш вышел в путь далёкий,
Сердцем горящим, духом смелым.
Шагает смело через леса,
Через поля, холмы и реки.

### Горы высокие вздымались,
Скалы грозили путнику взглядом.
Но герой шёл вперёд уверенно,
Не страшась преград ни днём, ни ночью.

### Лес шумел, деревья качали ветвями,
Ручьи звенели звонко и чисто.
Герой встретил мудрого старца,
Кто указал дорогу верно.

---

## Часть третья: Встреча с врагом

### Но враг коварный встал на пути,
Чёрная сила тёмной ночи.
Гневно рычал, меч обнажая,
Готов был бой жестокий нача

❓ For a hint, run the cell below!

In [49]:
from hints import exercise_2_3_hint; print(exercise_2_3_hint)

The grading function in this cell is looking for a response that is equal to or greater than 800 words.
Because LLMs aren't great at counting words yet, you may have to overshoot your target.


---

## Part 3

Because GigaChat has no context beyond the prompt, it is often important to **set a specific role (including necessary context)**. This is known as role prompting; more detailed role context generally improves results.

**Priming GigaChat with a role can improve GigaChat's performance** in a variety of fields, from writing to coding to summarizing. It's like how humans can sometimes be helped when told to "think like a ______". Role prompting can also change the style, tone, and manner of GigaChat's response.

**Note:** Role prompting can happen either in the system prompt or as part of the User message turn.

### Examples

In the example below, we see that without role prompting, GigaChat provides a **straightforward and non-stylized answer** when asked to give a single sentence perspective on skateboarding.

However, when we prime GigaChat to inhabit the role of a cat, GigaChat's perspective changes, and thus **GigaChat's response tone, style, content adapts to the new role**. 

**Note:** A bonus technique is to **provide GigaChat context on its intended audience**. The prompt can also specify whom GigaChat is speaking to (e.g., "You are a cat" vs "you are a cat talking to a crowd of skateboarders").

Here is the prompt without role prompting in the system prompt:

In [51]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print GigaChat's response
print(get_completion(PROMPT))

Skateboarding is an exhilarating, creative and versatile sport that combines physical challenge, artistic expression and a vibrant culture rooted in freedom and individuality.


Here is the same user question, except with role prompting.

In [52]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print GigaChat's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Skateboarding is fun because it lets me glide and jump around like I'm flying through the air!


it is possible to use role prompting as a way to get GigaChat to emulate certain styles in writing, speak in a certain voice, or guide the complexity of its answers. **Role prompting can also make GigaChat better at performing math or logic tasks.**

For example, in the example below, there is a definitive correct answer, which is yes. However, GigaChat gets it wrong and thinks it lacks information, which it doesn't:

In [53]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print GigaChat's response
print(get_completion(PROMPT))

### Анализ ситуации

1. **Джек** женат (женат).
2. **Анна** — её семейное положение неизвестно.
3. **Георгий** не женат (не замужем).

- Джек смотрит на Анну.
- Анна смотрит на Джорджа.

### Вопрос: 
«Смотрит ли женатый человек на человека, который не женат?»

#### Рассмотрим возможные варианты:

1. **Вариант 1:**  
   Если Анна замужем, тогда она смотрит на Джорджа, который не женат. В таком случае да, женатый человек смотрит на человека, который не женат.
   
2. **Вариант 2:**  
   Если Анна не замужем, тогда ситуация становится сложнее. Но даже в этом случае мы знаем, что Джека интересует именно её состояние, а не Джорджа. То есть, независимо от семейного положения Анны, вопрос сводится к тому, смотрит ли женатый человек на кого-то, кто не женат.

### Итоговый вывод:

Независимо от того, замужем ли Анна, факт остаётся неизменным: женатый человек (Джек) смотрит на другого человека (Анну), а та, в свою очередь, смотрит на человека, который не женат (Георгия). Таким образом, ответ одно

At this point, what if we **prime GigaChat to act as a logic bot**? How will that change GigaChat's answer? 

It turns out that with this new role assignment, GigaChat gets it right. (Although notably not for all the right reasons)

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print GigaChat's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Let's break down the information given:

1. **Jack** is looking at **Anne**.
2. **Anne** is looking at **George**.
3. **Jack** is married.
4. **George** is not married (unmarried).
5. We do not know whether **Anne** is married or not.

We need to determine if a married person is looking at an unmarried person.

- Since **Jack** is married and he is looking at **Anne**, we can say that a married person (**Jack**) is potentially looking at someone who might be either married or unmarried (**Anne**).
  
- However, we don't have enough information about Anne's marital status. Therefore, we cannot definitively conclude that Anne is unmarried based on the provided information.

- **Anne** is looking at **George**, who is unmarried. If Anne is married, then she would be a married person looking at an unmarried person.

Since we can't confirm Anne’s marital status, we must consider both possibilities:

1. If Anne is married: A married person (Anne) is looking at an unmarried person (George).
2

---

## Exercises
- [Exercise 3.1 - Math Correction](#exercise-31---math-correction)

### Exercise 3.1 - Math Correction
In some instances, **GigaChat may struggle with mathematics**, even simple mathematics. Below, GigaChat incorrectly assesses the math problem as correctly solved, even though there's an obvious arithmetic mistake in the second step. Note that GigaChat actually catches the mistake when going through step-by-step, but doesn't jump to the conclusion that the overall solution is wrong.

Modify the `PROMPT` and / or the `SYSTEM_PROMPT` to make GigaChat grade the solution as `incorrectly` solved, rather than correctly solved. 


In [57]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# Get GigaChat's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print GigaChat's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

Да, решение верно.

Рассмотрим пошагово:

1. Исходное уравнение:  
$$2x - 3 = 9$$

2. Добавляем 3 к обеим сторонам уравнения (чтобы избавиться от числа "-3"):  
$$2x - 3 + 3 = 9 + 3$$  
$$2x = 12$$

3. Делим обе стороны уравнения на 2 (чтобы найти значение переменной x):  
$$\frac{2x}{2} = \frac{12}{2}$$  
$$x = 6 : 2$$  
$$x = 3$$

Таким образом, правильный ответ действительно $x=3$.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: False


❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_3_1_hint; print(exercise_3_1_hint)

---

## Part 4

Oftentimes, full prompts are not desired; instead, **prompt templates can be modified later with additional input data before submitting to GigaChat**. This is useful when the same task is repeated with different input data.

Luckily, we can do this pretty easily by **separating the fixed skeleton of the prompt from variable user input, then substituting the user input into the prompt** before sending the full prompt to GigaChat. 

The following section'll walk step by step through how to write a substitutable prompt template, as well as how to substitute in user input.

### Examples

In this first example, we're asking GigaChat to act as an animal noise generator. Notice that the full prompt submitted to GigaChat is just the `PROMPT_TEMPLATE` substituted with the input (in this case, "Cow"). Notice that the word "Cow" replaces the `ANIMAL` placeholder via an f-string when we print out the full prompt.

**Note:** The placeholder variable name is arbitrary in practice. The placeholder is named `ANIMAL` in this example; alternatives such as `CREATURE` or `A` would also work. Using specific, relevant variable names keeps the prompt template readable without substitution. Ensure the chosen variable name matches the prompt template f-string.

In [62]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
I will tell you the name of an animal. Please respond with the noise that animal makes. Cow

------------------------------------- GigaChat's response -------------------------------------
Мoo (муу)


Why would we want to separate and substitute inputs like this? Well, **prompt templates simplify repetitive tasks**. This section demonstrates a prompt structure that invites third party users to submit content to the prompt (in this case, the animal whose sound is generated). These third party users don't have to write or even see the full prompt. All they have to do is fill in variables.

We do this substitution here using variables and f-strings, but it is possible to also do it with the format() method.

**Note:** Prompt templates can have as many variables as desired!

When introducing substitution variables like this, it is very important to **make sure GigaChat knows where variables start and end** (vs. instructions or task descriptions). This section demonstrates look at an example where there is no separation between the instructions and the substitution variable.

To our human eyes, it is very clear where the variable begins and ends in the prompt template below. However, in the fully substituted prompt, that delineation becomes unclear.

In [65]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo GigaChat. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Yo GigaChat. Show up at 6am tomorrow because I'm the CEO and I say so. <----- Make this email more polite but don't change anything else about it.

------------------------------------- GigaChat's response -------------------------------------
Вот корректированный вариант письма, сохранивший суть, но сделанный вежливее:

---

**Subject:** Meeting at 6:00 AM Tomorrow

> Yo GigaChat,
>
> Please be present for our meeting at **6:00 AM** tomorrow. As the CEO, I ask that you make this a priority.
>
> Thanks!

---  

Такого стиля достаточно, чтобы сохранить необходимую строгость тона, но сделать обращение более уважительным и корректным.


Here, **GigaChat thinks "Yo GigaChat" is part of the email it's supposed to rewrite**! it is possible to tell because it begins its rewrite with "Dear GigaChat". To the human eye, it's clear, particularly in the prompt template where the email begins and ends, but it becomes much less clear in the prompt after substitution.

How do we solve this? **Wrap the input in XML tags**! We did this below, and as it is possible to see, there's no more "Dear GigaChat" in the output.

[XML tags](https://developers.sber.ru/help/gigachat-api/prompt-engineering) are angle-bracket tags like `<tag></tag>`. They come in pairs and consist of an opening tag, such as `<tag>`, and a closing tag marked by a `/`, such as `</tag>`. XML tags are used to wrap around content, like this: `<tag>content</tag>`.

**Note:** While GigaChat can recognize a wide range of separators and delimiters, XML tags are recommended as separators because GigaChat was trained to recognize them as a prompt organizing mechanism. Outside of function calling, there are no special XML tags that meaningfully boost performance.

In [64]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo GigaChat. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Yo GigaChat. <email>Show up at 6am tomorrow because I'm the CEO and I say so.</email> <----- Make this email more polite but don't change anything else about it.

------------------------------------- GigaChat's response -------------------------------------
**Subject:** Meeting Tomorrow Morning  

**Email:**

> Good morning,  
> 
> Could you kindly join us for a meeting at **6 am** tomorrow? As the CEO, I’d appreciate your presence.  
> 
> Thank you in advance for your cooperation.  
> 
> Best regards,  
> [Your Name]


This section demonstrates see another example of how XML tags can help us. 

In the following prompt, **GigaChat incorrectly interprets what part of the prompt is the instruction vs. the input**. It incorrectly considers `Each is about an animal, like rabbits` to be part of the list due to the formatting, when the user (the one filling out the `SENTENCES` variable) presumably did not want that.

In [75]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs

------------------------------------- GigaChat's response -------------------------------------
Second item on the list:  
**This sentence may appear to be about dogs but it's actually about pigs.**


To fix this, we just need to **surround the user input sentences in XML tags**. This shows GigaChat where the input data begins and ends despite the misleading hyphen before `Each is about an animal, like rabbits.`

In [72]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
 Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs
</sentences>

------------------------------------- GigaChat's response -------------------------------------
**Second item on the list:**

This sentence is about spiders


---

## Exercises
- [Exercise 4.1 - Haiku Topic](#exercise-41---haiku-topic)
- [Exercise 4.2 - Dog Question with Typos](#exercise-42---dog-question-with-typos)
- [Exercise 4.3 - Dog Question Part 2](#exercise-42---dog-question-part-2)

### Exercise 4.1 - Haiku Topic
Modify the `PROMPT` so that it's a template that will take in a variable called `TOPIC` and output a haiku about the topic. This exercise tests understanding of the variable templating structure with f-strings.

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
Let's talk about the Pigs

------------------------------------- GigaChat's response -------------------------------------
О чём конкретно ты хочешь поговорить про свиней (pigs)? Вот несколько возможных направлений:

1. **Свиньи как домашние животные**: особенности содержания, уход, породы, характер.
2. **Пищевые свиньи**: разведение, выращивание, особенности питания, порода мясных свиней.
3. **Экологические аспекты**: роль свиней в экосистеме, влияние на окружающую среду, проблемы животноводства.
4. **Культурные и исторические аспекты**: символика свиньи в разных культурах, мифология, традиции.
5. **Научные исследования**: генетические особенности, болезни, селекция, научные открытия, связанные со свиньями.
6. **Юмор и культура**: анекдоты, пословицы, поговорки, шутки про свиней.
7. **Экономика и бизнес**: рынок свинины, производство мяса, переработка продукции.
8. **Здоровье человека**: связ

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### Exercise 4.2 - Dog Question with Typos
Fix the `PROMPT` by adding XML tags so that GigaChat produces the right answer. 

Try not to change anything else about the prompt. The messy and mistake-ridden writing is intentional, so it is possible to see how GigaChat reacts to such mistakes.

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ For a hint, run the cell below!

In [77]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

The grading function in this exercise is looking for a response that includes the word "brown".
If you surround "{QUESTION}" in XML tags, how does that change Claude's response?


### Exercise 4.3 - Dog Question Part 2
Fix the `PROMPT` **WITHOUT** adding XML tags. Instead, remove only one or two words from the prompt.

Just as with the above exercises, try not to change anything else about the prompt. This demonstrates the kinds of language GigaChat can parse and understand.

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ For a hint, run the cell below!

In [3]:
from hints import exercise_4_3_hint; print(exercise_4_3_hint)

The grading function in this exercise is looking for a response that includes the word "brown".
Try removing one word or section of characters at a time, starting with the parts that make the least sense. Doing this one word at a time will also help you see just how much GigaChat can or can't parse and understand.


---

## Part 5

**GigaChat can format its output in a wide variety of ways**. It often suffices to ask explicitly.

One of these ways is by using XML tags to separate out the response from any other superfluous text. This material already shows that XML tags can make prompts clearer and more parseable to GigaChat. It turns out, it is possible to also ask GigaChat to **use XML tags to make its output clearer and more easily understandable** to humans.

### Examples

Remember the 'poem preamble problem' we solved in Part 2 by asking GigaChat to skip the preamble entirely? It turns out we can also achieve a similar outcome by **telling GigaChat to put the poem in XML tags**.

In [3]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in <haiku> tags.

------------------------------------- GigaChat's response -------------------------------------
```
<haiku>
В норке тихо спит зайчонок,
Солнце прыгнуло в траву —
Короток сон в полдень жаркий.
</haiku>


Why is this something we'd want to do? Well, having the output in **XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags**.

An extension of this technique is to **put the first XML tag in the `assistant` turn**. Placing text in the `assistant` turn indicates that GigaChat has already said something and should continue from that point onward. This technique is called "speaking for GigaChat" or "prefilling GigaChat's response."

The following section've done this with the first `<haiku>` XML tag. Notice how GigaChat continues directly from where we left off.

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for GigaChat's response
PREFILL = "<haiku>"

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

GigaChat also supports other output formatting styles, notably `JSON`. To encourage JSON output (not deterministically), prefill GigaChat's response with the opening bracket, `{`}.

In [4]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for GigaChat's response
PREFILL = "{"

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN
{

------------------------------------- GigaChat's response -------------------------------------
  
  "first_line": "Мягко ступает лапкой”,  
  "second_line": “Тихо мурлычет рядом”,  
  "third_line": “Солнце греет окна”  
}


Below is an example of **multiple input variables in the same prompt AND output formatting specification, all done using XML tags**.

In [5]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey GigaChat. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for GigaChat's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey GigaChat. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.

ASSISTANT TURN
<olde english_email>

------------------------------------- GigaChat's response -------------------------------------
  
  <greeting>Hail, good sir Zack!</greeting>  
  <body>  
   Nay, I but lightly touch upon thee, to enquire after that certain prompt thou wert charged with composing. </body>  
  <closing>Yours most courteously,</closing>  
</olde english_email>


---

## Exercises
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT
Forced to make a choice, GigaChat designates Michael Jordan as the best basketball player of all time. Can we get GigaChat to pick someone else?

Change the `PREFILL` variable to **compell GigaChat to make a detailed argument that the best basketball player of all time is Stephen Curry**. Try not to change anything except `PREFILL` as that is the focus of this exercise.

In [24]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for GigaChat's response
PREFILL = ""

# Get GigaChat's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Who is the best basketball player of all time? Please choose one specific player.

ASSISTANT TURN


------------------------------------- GigaChat's response -------------------------------------
Выбор лучшего баскетболиста всех времен — субъективная тема, зависящая от множества факторов: стиля игры, достижений, влияния на спорт и даже личных предпочтений болельщиков. Однако общепринятым выбором среди экспертов считается **Майкл Джордан** (Michael Jordan).

### Почему именно Майкл Джордан?
- **Достижения**: За карьеру в НБА Джордан выиграл шесть титулов чемпиона, десять раз становился MVP регулярного чемпионата, девять раз побеждал в номинации MVP плей-офф и дважды признавался самым ценным игроком финала НБА.
- **Стиль игры**: Его техника броска, умение контролировать игру и способность вести команду к победам сделали его иконой баскетбола.
- **Легендарность**: Джордан стал символом 

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### Exercise 5.2 - Two Haikus
Modify the `PROMPT` below using XML tags so that GigaChat writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [ ]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for GigaChat's response
PREFILL = "<haiku>"

# Get GigaChat's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### Exercise 5.3 - Two Haikus, Two Animals
Modify the `PROMPT` below so that **GigaChat produces two haikus about two different animals**. Use `{ANIMAL1}` as a stand-in for the first substitution, and `{ANIMAL2}` as a stand-in for the second substitution.

In [ ]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ For a hint, run the cell below!

In [2]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

The grading function in this exercise is looking for a response that contains the words "tail", "cat", and "<haiku>".
It's helpful to break this exercise down to several steps.								
1.	Modify the initial prompt template so that GigaChat writes two poems.							
2.	Give GigaChat indicators as to what the poems will be about, but instead of writing in the subjects directly (e.g., dog, cat, etc.), replace those subjects with the keywords "{ANIMAL1}" and "{ANIMAL2}".							
3.	Run the prompt and make sure that the full prompt with variable substitutions has all the words correctly substituted. If not, check to make sure your {bracket} tags are spelled correctly and formatted correctly with single moustache brackets.


---

## Part 6

If someone were asked to answer several complicated questions immediately after waking, the response would likely be inferior to one produced with time for deliberation.

GigaChat is the same way.

**Giving GigaChat time to think step by step sometimes makes GigaChat more accurate**, particularly for complex tasks. However, **thinking only counts when it's out loud**. It is not possible to require reasoning while outputting only the final answer; in that case no reasoning is actually performed.

### Examples

In the prompt below, it's clear to a human reader that the second sentence belies the first. But **GigaChat takes the word "unrelated" too literally**.

In [10]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print GigaChat's response
print(get_completion(PROMPT))

### Arguments

1. **Positive Phrases:**
   - "blew my mind"
   - "freshness and originality"
   
2. **Negative Phrase:**
   - "living under a rock since the year 1900” (this is sarcasm indicating that the reviewer was unaware of recent developments)

3. **Contextual Analysis:**
   - The phrase "under a rock" typically implies being out-of-touch or uninformed about current events. However, in this context, it’s used humorously to emphasize how impressed the reviewer was by the film's uniqueness.

4. **Overall Tone:**
   - Despite the sarcastic remark, the overall tone leans towards positivity due to the strong praise for the film's freshness and originality.

### Final Answer

The movie review has a **positive sentiment**.


To improve GigaChat's response, let's **allow GigaChat to think things out first before answering**. We do that by literally spelling out the steps that GigaChat should take in order to process and think through its task. Along with a dash of role prompting, this empowers GigaChat to understand the review more deeply.

In [13]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> firstly and <negative-argument> secondary XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print GigaChat's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

<positive-argument>
1. The reviewer describes the movie as "blow[ing] [their] mind" which indicates a strong positive reaction to the film.
2. The use of the word "freshness" suggests that the movie brought something new and exciting to the genre.
3. The term "originality" implies that the movie stood out from other films, adding another layer of praise.

<negative-argument>
1. The phrase "living under a rock since 1900" is a humorous way of saying they were unaware of recent events or trends, implying that their comparison might not be entirely fair or accurate.
2. This could suggest that the reviewer may not be an expert on current cinema trends, potentially affecting the credibility of their opinion.

Answer: Positive


**GigaChat is sometimes sensitive to ordering**. This example is on the frontier of GigaChat's ability to understand nuanced text, and when we swap the order of the arguments from the previous example so that negative is first and positive is second, this changes GigaChat's overall assessment to positive.

In most situations (but not all, confusingly enough), **GigaChat is more likely to choose the second of two options**, possibly because in its training data from the web, second options were more likely to be correct.

In [15]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> firstly and <positive-argument> negative XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print GigaChat's response
print(get_completion(PROMPT))

**<negative-argument>**

1. **Tone and Context**: The phrase “living under a rock since 1900” is typically used to indicate that someone has been out of touch with modern culture or society. This can imply that the reviewer feels disconnected from contemporary cinema or entertainment trends.
   
2. **Unrelated Comment**: The comment about living under a rock seems irrelevant to the actual content of the movie being reviewed. It may suggest that the reviewer found it difficult to connect with the film’s themes or narrative.

3. **Potential Negative Implication**: If the reviewer felt so isolated from current cinematic norms, they might not have fully appreciated the movie's unique qualities, leading to an overall negative perception.

**<positive-argument>**

1. **Positive Language**: The use of phrases like “blew my mind” and “freshness and originality” indicates strong enthusiasm and appreciation for the movie’s uniqueness.
   
2. **High Praise**: Describing something as blowing one’s

**Letting GigaChat think can shift GigaChat's answer from incorrect to correct**. It's that simple in many cases where GigaChat makes mistakes!

This section demonstrates go through an example where GigaChat's answer is incorrect to see how asking GigaChat to think can fix that.

In [16]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print GigaChat's response
print(get_completion(PROMPT))

One famous movie starring an actor born in 1956 is **"The Silence of the Lambs"** (1991), with Anthony Hopkins playing the iconic role of Hannibal Lecter.


This section demonstrates fix this by asking GigaChat to think step by step, this time in `<brainstorm>` tags.

In [17]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print GigaChat's response
print(get_completion(PROMPT))

<brainstorm>

- **Tom Cruise** — born July 3, 1962 (not 1956)
- **Johnny Depp** — born June 9, 1963 (not 1956)
- **Denzel Washington** — born December 28, 1954 (not 1956)
- **Al Pacino** — born April 25, 1944 (not 1956)
- **Robert Downey Jr.** — born April 4, 1965 (not 1956)
- **Christian Bale** — born January 30, 1974 (not 1956)
- **Russell Crowe** — born April 4, 1964 (not 1956)
- **Matt Damon** — born October 8, 1970 (not 1956)
- **Leonardo DiCaprio** — born November 11, 1974 (not 1956)
- **Daniel Craig** — born March 2, 1972 (not 1956)
- **Jake Gyllenhaal** — born February 3, 1980 (not 1956)
- **Ryan Gosling** — born November 12, 1980 (not 1956)
- **Mark Wahlberg** — born October 17, 1971 (not 1956)
- **Keanu Reeves** — born September 2, 1964 (not 1956)
- **Ewan McGregor** — born May 31, 1971 (not 1956)
- **Clive Owen** — born August 10, 1964 (not 1956)
- **Colin Farrell** — born July 6, 1976 (not 1956)
- **Hugh Jackman** — born October 12, 1968 (not 1956)
- **Chris Hemsworth** — b

---

## Exercises
- [Exercise 6.1 - Classifying Emails](#exercise-61---classifying-emails)
- [Exercise 6.2 - Email Classification Formatting](#exercise-62---email-classification-formatting)

### Exercise 6.1 - Classifying Emails
In this exercise, we'll be instructing GigaChat to sort emails into the following categories:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)

For the first part of the exercise, change the `PROMPT` to **make GigaChat output the correct classification and ONLY the classification**. Answers must **include the letter (A - D) of the correct choice, with parentheses, and the name of the category**.

Refer to the comments beside each email in the `EMAILS` list to know which category that email should be classified under.

In [21]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for GigaChat's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get GigaChat's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade GigaChat's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print GigaChat's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- GigaChat's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please classify this email as either green or blue: Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.

ASSISTANT TURN


------------------------------------- GigaChat's response -------------------------------------
**Classification: Blue**

The email describes an issue with the equipment (Mixmaster4000), mentions unusual symptoms (strange noises, smoky smell, plastic-like burning odor), and requests a replacement. This indicates that it's related to technical support or maintenance issues, which typically fall under the category of **blue communication**.

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: False 






--------------------------- Full prompt with variable substutions ------

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

Still stuck? The cell can be executed below for an example solution.						

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### Exercise 6.2 - Email Classification Formatting
In this exercise, we're going to refine the output of the above prompt to yield an answer formatted exactly how we want it. 

Use an output-formatting technique to make GigaChat wrap only the letter of the correct classification in `<answer></answer>` tags. For instance, the answer to the first email should contain the exact string `<answer>B</answer>`.

Refer to the comments beside each email in the `EMAILS` list if needed to confirm the correct letter category.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for GigaChat's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get GigaChat's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade GigaChat's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print GigaChat's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- GigaChat's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

---

## Part 7

**Providing GigaChat examples of desired or undesired behavior is extremely effective** for:
- Getting the right answer
- Getting the answer in the right format

This sort of prompting is also called "**few shot prompting**". The terms "zero-shot", "n-shot", and "one-shot" are also common. The number of "shots" refers to how many examples are used within the prompt.

### Examples

Example: a developer building a "parent bot" that responds to questions from kids. **GigaChat's default response is quite formal and robotic**, which is undesirable in that setting.

In [26]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print GigaChat's response
print(get_completion(PROMPT))

Да, если ты хорошо себя вёл весь год (или хотя бы большую его часть), Дед Мороз обязательно заглянет в твой дом на Рождество и оставит подарки под ёлкой. Но помни: чтобы всё получилось, важно верить в чудо и оставить угощение для Деда Мороза — печенье, яблоко или молоко. А ещё лучше — написать письмо заранее, чтобы рассказать ему о своих желаниях!


Describing the desired tone is possible, but it is often easier to **provide GigaChat with a few examples of ideal responses**.

In [27]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print GigaChat's response
print(get_completion(PROMPT))

A: Yes, honey — if you've been good all year! Make sure to leave cookies and milk out for Santa, and don't forget to hang up your stocking by the fireplace.


In the following formatting example, we could walk GigaChat step by step through a set of formatting instructions on how to extract names and professions and then format them exactly the way we want, or we could just **provide GigaChat with some correctly-formatted examples and GigaChat can extrapolate from there**. Note the `<individuals>` in the `assistant` turn to start GigaChat off on the right foot.

In [53]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.

<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for GigaChat's response
PREFILL = "<individuals>"

SYSTEM_PROMPT="Provide the answer in specific format. Examples of question and answers"

# Print GigaChat's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT, SYSTEM_PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.

<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND 

---

## Exercises
- [Exercise 7.1 - Email Formatting via Examples](#exercise-71---email-formatting-via-examples)

### Exercise 7.1 - Email Formatting via Examples
We're going to redo Exercise 6.2, but this time, we're going to edit the `PROMPT` to use "few-shot" examples of emails + proper classification (and formatting) to get GigaChat to output the correct answer. We want the *last* letter of GigaChat's output to be the letter of the category.

Refer to the comments beside each email in the `EMAILS` list if needed to confirm the correct letter category.

Remember that these are the categories for the emails:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)								

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for GigaChat's response
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get GigaChat's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade GigaChat's response
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Print GigaChat's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- GigaChat's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

Still stuck? The cell can be executed below for an example solution.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

---

## Part 8

Some bad news: **GigaChat sometimes "hallucinates" and makes claims that are untrue or unjustified**. The good news: there are techniques it is possible to use to minimize hallucinations.
				
The following section'll go over a few of these techniques, namely:
- Giving GigaChat the option to say it doesn't know the answer to a question
- Asking GigaChat to find evidence before answering

However, **there are many methods to avoid hallucinations**, including techniques already covered. If hallucinations occur, experiment with multiple techniques to improve accuracy.

### Examples

Here is a question about general factual knowledge in answer to which **GigaChat hallucinates several large hippos because it's trying to be as helpful as possible**.

In [34]:
# Prompt
PROMPT = "Who is the heaviest hippo of all time?"

# Print GigaChat's response
print(get_completion(PROMPT))

The heaviest known hippopotamus on record was a male hippo named **Tonkie**, who lived in South Africa during the early 1980s. He weighed an incredible **4,750 kg (approximately 10,460 pounds)** at his peak weight.

This remarkable size made him one of the largest land mammals ever recorded outside of elephants and rhinoceroses. Tonkie was kept at the Kruger National Park's Orpen Game Reserve, where he became famous for his massive girth and gentle nature despite his intimidating appearance.


A solution we can try here is to "**give GigaChat an out**" — tell GigaChat that it's OK for it to decline to answer, or to only answer if it actually knows the answer with certainty.

In [31]:
# Prompt
PROMPT = "Who is the heaviest hippo of all time? Only answer if you know the answer with certainty."

# Print GigaChat's response
print(get_completion(PROMPT))

На сегодняшний день достоверной информации о самом тяжелом когда-либо зарегистрированном гиппопотаме нет. Однако известно, что взрослые самцы африканского бегемота (Hippopotamus amphibius), самого крупного вида гиппопотамов, обычно весят от 1 500 до 3 200 кг. Самки немного легче — около 1 300–1 900 кг.

Самый крупный официально зафиксированный бегемот был убит в Южной Африке в конце XIX века и весил примерно **4 700 кг**. Этот случай задокументирован в исторических источниках, однако официального подтверждения веса именно этого животного нет.

Таким образом, самый тяжелый гиппопотам в истории — это скорее исторический факт, подтвержденный лишь косвенными свидетельствами, а не документально подтвержденная информация.


In the prompt below, we give GigaChat a long document containing some "distractor information" that is almost but not quite relevant to the user's question. **Without prompting help, GigaChat falls for the distractor information** and gives an incorrect "hallucinated" answer as to the size of Matterport's subscriber base as of May 31, 2020.

**Note:** A later chapter presents it as best practice to place the question at the bottom *after* any text or document; it is placed at the top here for readability. Optionally, double click on the prompt cell to view the full prompt text (it is long!).

In [32]:
# Prompt
PROMPT = """<question>What was Matterport's subscriber base on the precise date of May 31, 2020?</question>
Please read the below document. Then write a brief numerical answer inside <answer> tags.

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterport is leading the digitization and datafication of the built world. We believe the digital transformation of the built world will fundamentally change the way people interact with buildings and the physical spaces around them.
Since its founding in 2011, Matterport’s pioneering technology has set the standard for digitizing, accessing and managing buildings, spaces and places online. Our platform’s innovative software, spatial data-driven data science, and 3D capture technology have broken down the barriers that have kept the largest asset class in the world, buildings and physical spaces, offline and underutilized for many years. We believe the digitization and datafication of the built world will continue to unlock significant operational efficiencies and property values, and that Matterport is the platform to lead this enormous global transformation.
The world is rapidly moving from offline to online. Digital transformation has made a powerful and lasting impact across every business and industry today. According to International Data Corporation, or IDC, over $6.8 trillion of direct investments will be made on digital transformation from 2020 to 2023, the global digital transformation spending is forecasted to reach $3.4 trillion in 2026 with a five-year compound annual growth rate (“CAGR”) of 16.3%, and digital twin investments are expected to have a five-year CAGR of 35.2%. With this secular shift, there is also growing demand for the built world to transition from physical to digital. Nevertheless, the vast majority of buildings and spaces remain offline and undigitized. The global building stock, estimated by Savills to be $327 trillion in total property value as of 2021, remains largely offline today, and we estimate that less than 0.1% is penetrated by digital transformation.
Matterport was among the first to recognize the increasing need for digitization of the built world and the power of spatial data, the unique details underlying buildings and spaces, in facilitating the understanding of buildings and spaces. In the past, technology advanced physical road maps to the data-rich, digital maps and location services we all rely on today. Matterport now digitizes buildings, creating a data-rich environment to vastly increase our understanding and the full potential of each and every space we capture. Just as we can instantly, at the touch of a button, learn the fastest route from one city to another or locate the nearest coffee shops, Matterport’s spatial data for buildings unlocks a rich set of insights and learnings about properties and spaces worldwide. In addition, just as the geo-spatial mapping platforms of today have opened their mapping data to industry to create new business models such as ridesharing, e-commerce, food delivery marketplaces, and even short-term rental and home sharing, open access to Matterport’s structured spatial data is enabling new opportunities and business models for hospitality, facilities management, insurance, construction, real estate and retail, among others.
We believe the total addressable market opportunity for digitizing the built world is over $240 billion, and could be as high as $1 trillion as the market matures at scale. This is based on our analysis, modeling and understanding of the global building stock of over 4 billion properties and 20 billion spaces in the world today. With the help of artificial intelligence (“AI”), machine learning (“ML”) and deep learning (“DL”) technologies, we believe that, with the additional monetization opportunities from powerful spatial data-driven property insights and analytics, the total addressable market for the digitization and datafication of the built world will reach more than $1 trillion.

Our spatial data platform and capture of digital twins deliver value across a diverse set of industries and use cases. Large retailers can manage thousands of store locations remotely, real estate agencies can provide virtual open houses for hundreds of properties and thousands of visitors at the same time, property developers can monitor the entirety of the construction process with greater detail and speed, and insurance companies can more precisely document and evaluate claims and underwriting assessments with efficiency and precision. Matterport delivers the critical digital experience, tools and information that matter to our subscribers about properties of virtually any size, shape, and location worldwide.
For nearly a decade, we have been growing our spatial data platform and expanding our capabilities in order to create the most detailed, accurate, and data-rich digital twins available. Moreover, our 3D reconstruction process is fully automated, allowing our solution to scale with equal precision to millions of buildings and spaces of any type, shape, and size in the world. The universal applicability of our service provides Matterport significant scale and reach across diverse verticals and any geography. As of December 31, 2022, our subscriber base had grown approximately 39% to over 701,000 subscribers from 503,000 subscribers as of December 31, 2021, with our digital twins reaching more than 170 countries. We have digitized more than 28 billion square feet of space across multiple industries, representing significant scale and growth over the rest of the market.

As we continue to transform buildings into data worldwide, we are extending our spatial data platform to further transform property planning, development, management and intelligence for our subscribers across industries to become the de facto building and business intelligence engine for the built world. We believe the demand for spatial data and resulting insights for enterprises, businesses and institutions across industries, including real estate, architecture, engineering and construction (“AEC”), retail, insurance and government, will continue to grow rapidly.
We believe digitization and datafication represent a tremendous greenfield opportunity for growth across this massive category and asset class. From the early stages of design and development to marketing, operations, insurance and building repair and maintenance, our platform’s software and technology provide subscribers critical tools and insights to drive cost savings, increase revenues and optimally manage their buildings and spaces. We believe that hundreds of billions of dollars in unrealized utilization and operating efficiencies in the built world can be unlocked through the power of our spatial data platform. Our platform and data solutions have universal applicability across industries and building categories, giving Matterport a significant advantage as we can address the entirety of this large market opportunity and increase the value of what we believe to be the largest asset class in the world.
With a demonstrated track record of delivering value to our subscribers, our offerings include software subscription, data licensing, services and product hardware. As of December 31, 2022, our subscriber base included over 24% of Fortune 1000 companies, with less than 10% of our total revenue generated from our top 10 subscribers. We expect more than 80% of our revenue to come from our software subscription and data license solutions by 2025. Our innovative 3D capture products, the Pro2 and Pro3 Cameras, have played an integral part in shaping the 3D building and property visualization ecosystem. The Pro2 and Pro3 Cameras have driven adoption of our solutions and have generated the unique high-quality and scaled data set that has enabled Cortex, our proprietary AI software engine, to become the pioneering engine for digital twin creation. With this data advantage initially spurred by the Pro2 Camera, we have developed a capture device agnostic platform that scales and can generate new building and property insights for our subscribers across industries and geographies.
We have recently experienced rapid growth. Our subscribers have grown approximately 49-fold from December 31, 2018 to December 31, 2022. Our revenue increased by approximately 22% to $136.1 million for the year ended December 31, 2022, from approximately $111.2 million for the year ended December 31, 2021. Our gross profit decreased by $8.1 million or 14%, to $51.8 million for the year ended December 31, 2022, from $60.0 million for the year ended December 31, 2021, primarily attributable to certain disruptive and incremental costs due to the global supply chain constraints in fiscal year 2022. Our ability to retain and grow the subscription revenue generated by our existing subscribers is an important measure of the health of our business and our future growth prospects. We track our performance in this area by measuring our net dollar expansion rate from the same set of customers across comparable periods. Our net dollar expansion rate of 103% for the three months ended December 31, 2022 demonstrates the stickiness and growth potential of our platform.
Our Industry and Market Opportunity
Today, the vast majority of buildings and spaces remain undigitized. We estimate our current serviceable addressable market includes approximately 1.3 billion spaces worldwide, primarily from the real estate and travel and hospitality sectors. With approximately 9.2 million spaces under management as of December 31, 2022, we are continuing to penetrate the global building stock and expand our footprint across various end markets, including residential and commercial real estate, facilities management, retail, AEC, insurance and repair, and travel and hospitality. We estimate our total addressable market to be more than 4 billion buildings and 20 billion spaces globally, yielding a more than $240 billion market opportunity. We believe that as Matterport’s unique spatial data library and property data services continue to grow, this opportunity could increase to more than $1 trillion based on the size of the building stock and the untapped value creation available to buildings worldwide. The constraints created by the COVID-19 pandemic have only reinforced and accelerated the importance of our scaled 3D capture solution that we have developed for diverse industries and markets over the past decade.

Our Spatial Data Platform
Overview
Our technology platform uses spatial data collected from a wide variety of digital capture devices to transform physical buildings and spaces into dimensionally accurate, photorealistic digital twins that provide our subscribers access to previously unavailable building information and insights.
As a first mover in this massive market for nearly a decade, we have developed and scaled our industry-leading 3D reconstruction technology powered by Cortex, our proprietary AI-driven software engine that uses machine learning to recreate a photorealistic, 3D virtual representation of an entire building structure, including contents, equipment and furnishings. The finished product is a detailed and dynamic replication of the physical space that can be explored, analyzed and customized from a web browser on any device, including smartphones. The power to manage even large-scale commercial buildings is in the palm of each subscriber’s hands, made possible by our advanced technology and breakthrough innovations across our entire spatial data technology stack.
Key elements of our spatial data platform include:
•Bringing offline buildings online. Traditionally, our customers needed to conduct in-person site visits to understand and assess their buildings and spaces. While photographs and floor plans can be helpful, these forms of two-dimensional (“2D”) representation have limited information and tend to be static and rigid, and thus lack the interactive element critical to a holistic understanding of each building and space. With the AI-powered capabilities of Cortex, our proprietary AI software, representation of physical objects is no longer confined to static 2D images and physical visits can be eliminated. Cortex helps to move the buildings and spaces from offline to online and makes them accessible to our customers in real-time and on demand from anywhere. After subscribers scan their buildings, our visualization algorithms accurately infer spatial positions and depths from flat, 2D imagery captured through the scans and transform them into high- fidelity and precise digital twin models. This creates a fully automated image processing pipeline to ensure that each digital twin is of professional grade image quality.
•Driven by spatial data. We are a data-driven company. Each incremental capture of a space grows the richness and depth of our spatial data library. Spatial data represents the unique and idiosyncratic details that underlie and compose the buildings and spaces in the human- made environment. Cortex uses the breadth of the billions of data points we have accumulated over the years to improve the 3D accuracy of our digital twins. We help our subscribers pinpoint the height, location and other characteristics of objects in their digital twin. Our sophisticated algorithms also deliver significant commercial value to our subscribers by generating data-based insights that allow them to confidently make assessments and decisions about their properties. For instance, property developers can assess the amount of natural heat and daylight coming from specific windows, retailers can ensure each store layout is up to the same level of code and brand requirements, and factories can insure machinery layouts meet specifications and location guidelines. With approximately 9.2 million spaces under management as of December 31, 2022, our spatial data library is the clearinghouse for information about the built world.
•Powered by AI and ML. Artificial intelligence and machine learning technologies effectively utilize spatial data to create a robust virtual experience that is dynamic, realistic, interactive, informative and permits multiple viewing angles. AI and ML also make costly cameras unnecessary for everyday scans—subscribers can now scan their spaces by simply tapping a button on their smartphones. As a result, Matterport is a device agnostic platform, helping us more rapidly scale and drive towards our mission of digitizing and indexing the built world.
Our value proposition to subscribers is designed to serve the entirety of the digital building lifecycle, from design and build to maintenance and operations, promotion, sale, lease, insure, repair, restore, secure and finance. As a result, we believe we are uniquely positioned to grow our revenue with our subscribers as we help them to discover opportunities to drive short- and long-term return on investment by taking their buildings and spaces from offline to online across their portfolios of properties.
Ubiquitous Capture
Matterport has become the standard for 3D space capture. Our technology platform empowers subscribers worldwide to quickly, easily and accurately digitize, customize and manage interactive and dimensionally accurate digital twins of their buildings and spaces.
The Matterport platform is designed to work with a wide range of LiDAR, spherical, 3D and 360 cameras, as well as smartphones, to suit the capture needs of all of our subscribers. This provides the flexibility to capture a space of any size, scale, and complexity, at anytime and anywhere.
•Matterport Pro3 is our newest 3D camera that scans properties faster than earlier versions to help accelerate project completion. Pro3 provides the highest accuracy scans of both indoor and outdoor spaces and is designed for speed, fidelity, versatility and accuracy. Capturing 3D data up to 100 meters away at less than 20 seconds per sweep, Pro3’s ultra-fast, high-precision LiDAR sensor can run for hours and takes millions of measurements in any conditions.
•Matterport Pro2 is our proprietary 3D camera that has been used to capture millions of spaces around the world with a high degree of fidelity, precision, speed and simplicity. Capable of capturing buildings more than 500,000 square feet in size, it has become the camera of choice for many residential, commercial, industrial and large-scale properties.
•360 Cameras. Matterport supports a selection of 360 cameras available in the market. These affordable, pocket sized devices deliver precision captures with high fidelity and are appropriate for capturing smaller homes, condos, short-term rentals, apartments, and more. The spherical lens image capture technology of these devices gives Cortex robust, detailed image data to transform panoramas into our industry-leading digital twins.
•LEICA BLK360. Through our partnership with Leica, our 3D reconstruction technology and our AI powered software engine, Cortex, transform this powerful LiDAR camera into an ultra-precise capture device for creating Matterport digital twins. It is the solution of choice for AEC professionals when exacting precision is required.
•Smartphone Capture. Our capture apps are commercially available for both iOS and Android. Matterport’s smartphone capture solution has democratized 3D capture, making it easy and accessible for anyone to digitize buildings and spaces with a recent iPhone device since the initial introduction of Matterport for iPhone in May 2020. In April 2021, we announced the official release of the Android Capture app, giving Android users the ability to quickly and easily capture buildings and spaces in immersive 3D. In February 2022, we launched Matterport Axis, a motorized mount that holds a smartphone and can be used with the Matterport Capture app to capture 3D digital twins of any physical space with increased speed, precision, and consistency.
Cortex and 3D Reconstruction (the Matterport Digital Twin)
With a spatial data library, as of December 31, 2022, of approximately 9.2 million spaces under management, representing approximately 28 billion captured square feet of space, we use our advanced ML and DL technologies to algorithmically transform the spatial data we capture into an accurate 3D digital reproduction of any physical space. This intelligent, automated 3D reconstruction is made possible by Cortex, our AI-powered software engine that includes a deep learning neural network that uses our spatial data library to understand how a building or space is divided into floors and rooms, where the doorways and openings are located, and what types of rooms are present, such that those forms are compiled and aligned with dimensional accuracy into a dynamic, photorealistic digital twin. Other components of Cortex include AI-powered computer vision technologies to identify and classify the contents inside a building or space, and object recognition technologies to identify and segment everything from furnishings and equipment to doors, windows, light fixtures, fire suppression sprinklers and fire escapes. Our highly scalable artificial intelligence platform enables our subscribers to tap into powerful, enhanced building data and insights at the click of a button.

The Science Behind the Matterport Digital Twin: Cortex AI Highlights
Matterport Runs on Cortex
Cortex is our AI-powered software engine that includes a precision deep learning neural network to create digital twins of any building or space. Developed using our proprietary spatial data captured with our Pro2 and Pro3 cameras, Cortex delivers a high degree of precision and accuracy while enabling 3D capture using everyday devices.
Generic neural networks struggle with 3D reconstruction of the real world. Matterport-optimized networks deliver more accurate and robust results. More than just raw training data, Matterport’s datasets allow us to develop new neural network architectures and evaluate them against user behavior and real-world data in millions of situations.
•Deep learning: Connecting and optimizing the detailed neural network data architecture of each space is key to creating robust, highly accurate 3D digital twins. Cortex evaluates and optimizes each 3D model against Matterport’s rich spatial data aggregated from millions of buildings and spaces and the human annotations of those data provided by tens of thousands of subscribers worldwide. Cortex’s evaluative abilities and its data-driven optimization of 3D reconstruction yield consistent, high-precision results across a wide array of building configurations, spaces and environments.
•Dynamic 3D reconstruction: Creating precise 3D spatial data at scale from 2D visuals and static images requires a combination of photorealistic, detailed data from multiple viewpoints and millions of spaces that train and optimize Cortex’s neural network and learning capabilities for improved 3D reconstruction of any space. Cortex’s capabilities combined with real-time spatial alignment algorithms in our 3D capture technology create an intuitive “preview” of any work in progress, allowing subscribers to work with their content interactively and in real-time.
•Computer vision: Cortex enables a suite of powerful features to enhance the value of digital twins. These include automatic measurements for rooms or objects in a room, automatic 2D-from-3D high-definition photo gallery creation, auto face blurring for privacy protection, custom videos, walkthroughs, auto room labeling and object recognition.
•Advanced image processing: Matterport’s computational photography algorithms create a fully automated image processing pipeline to help ensure that each digital twin is of professional grade image quality. Our patented technology makes 3D capture as simple as pressing a single button. Matterport’s software and technology manage the remaining steps, including white balance and camera-specific color correction, high dynamic range tone mapping, de-noising, haze removal, sharpening, saturation and other adjustments to improve image quality.
Spatial Data and AI-Powered Insights
Every Matterport digital twin contains extensive information about a building, room or physical space. The data uses our AI-powered Cortex engine. In addition to the Matterport digital twin itself, our spatial data consists of precision building geometry and structural detail, building contents, fixtures and condition, along with high-definition imagery and photorealistic detail from many vantage points in a space. Cortex employs a technique we call deep spatial indexing. Deep spatial indexing uses artificial intelligence, computer vision and deep learning to identify and convey important details about each space, its structure and its contents with precision and fidelity. We have created a robust spatial data standard that enables Matterport subscribers to harness an interoperable digital system of record for any building.
In addition to creating a highly interactive digital experience for subscribers through the construction of digital twins, we ask ourselves two questions for every subscriber: (1) what is important about their building or physical space and (2) what learnings and insights can we deliver for this space? Our AI-powered Cortex engine helps us answer these questions using our spatial data library to provide aggregated property trends and operational and valuation insights. Moreover, as the Matterport platform ecosystem continues to expand, our subscribers, partners and other third-party developers can bring their own tools to further the breadth and depth of insights they can harvest from our rich spatial data layer.
Extensible Platform Ecosystem
Matterport offers the largest and most accurate library of spatial data in the world, with, as of December 31, 2022, approximately 9.2 million spaces under management and approximately 28 billion captured square feet. The versatility of our spatial data platform and extensive enterprise software development kit and application programming interfaces (“APIs”) has allowed us to develop a robust global ecosystem of channels and partners that extend the Matterport value proposition by geography and vertical market. We intend to continue to deploy a broad set of workflow integrations with our partners and their subscribers to promote an integrated Matterport solution across our target markets. We are also developing a third-party software marketplace to extend the power of our spatial data platform with easy-to-deploy and easy-to-access Matterport software add-ons. The marketplace enables developers to build new applications and spatial data mining tools, enhance the Matterport 3D experience, and create new productivity and property management tools that supplement our core offerings. These value-added capabilities created by third-party developers enable a scalable new revenue stream, with Matterport sharing the subscription and services revenue from each add-on that is deployed to subscribers through the online marketplace. The network effects of our platform ecosystem contributes to the growth of our business, and we believe that it will continue to bolster future growth by enhancing subscriber stickiness and user engagement.
Examples of Matterport add-ons and extensions include:
•Add-ons: Encircle (easy-to-use field documentation tools for faster claims processing); WP Matterport Shortcode (free Wordpress plugin that allows Matterport to be embedded quickly and easily with a Matterport shortcode), WP3D Models (WordPress + Matterport integration plugin); Rela (all-in-one marketing solution for listings); CAPTUR3D (all-in-one Content Management System that extends value to Matterport digital twins); Private Model Emded (feature that allows enterprises to privately share digital twins with a large group of employees on the corporate network without requiring additional user licenses); Views (new workgroup collaboration framework to enable groups and large organizations to create separate, permissions-based workflows to manage different tasks with different teams); and Guided Tours and Tags (tool to elevate the visitor experience by creating directed virtual tours of any commercial or residential space tailored to the interests of their visitors). We unveiled our private beta integration with Amazon Web Services (AWS) IoT TwinMaker to enable enterprise customers to seamlessly connect IoT data into visually immersive and dimensionally accurate Matterport digital twin.
•Services: Matterport ADA Compliant Digital Twin (solution to provide American Disability Act compliant digital twins) and Enterprise Cloud Software Platform (reimagined cloud software platform for the enterprise that creates, publishes, and manages digital twins of buildings and spaces of any size of shape, indoors or outdoors).
Our Competitive Strengths
We believe that we have a number of competitive strengths that will enable our market leadership to grow. Our competitive strengths include:
•Breadth and depth of the Matterport platform. Our core strength is our all-in-one spatial data platform with broad reach across diverse verticals and geographies such as capture to processing to industries without customization. With the ability to integrate seamlessly with various enterprise systems, our platform delivers value across the property lifecycle for diverse end markets, including real estate, AEC, travel and hospitality, repair and insurance, and industrial and facilities. As of December 31, 2022, our global reach extended to subscribers in more than 170 countries, including over 24% of Fortune 1000 companies.
•Market leadership and first-mover advantage. Matterport defined the category of digitizing and datafying the built world almost a decade ago, and we have become the global leader in the category. As of December 31, 2022, we had over 701,000 subscribers on our platform and approximately 9.2 million spaces under management. Our leadership is primarily driven by the fact that we were the first mover in digital twin creation. As a result of our first mover advantage, we have amassed a deep and rich library of spatial data that continues to compound and enhance our leadership position.
•Significant network effect. With each new capture and piece of data added to our platform, the richness of our dataset and the depth of insights from our spaces under management grow. In addition, the combination of our ability to turn data into insights with incremental data from new data captures by our subscribers enables Matterport to develop features for subscribers to our platform. We were a first mover in building a spatial data library for the built world, and our leadership in gathering and deriving insights from data continues to compound and the relevance of those insights attracts more new subscribers.
•Massive spatial data library as the raw material for valuable property insights. The scale of our spatial data library is a significant advantage in deriving insights for our subscribers. Our spatial data library serves as vital ground truth for Cortex, enabling Matterport to create powerful 3D digital twins using a wide range of camera technology, including low-cost digital and smartphone cameras. As of December 31, 2022, our data came from approximately 9.2 million spaces under management and approximately 28 billion captured square feet. As a result, we have taken property insights and analytics to new levels, benefiting subscribers across various industries. For example, facilities managers significantly reduce the time needed to create building layouts, leading to a significant decrease in the cost of site surveying and as-built modeling. AEC subscribers use the analytics of each as-built space to streamline documentation and collaborate with ease.
•Global reach and scale. We are focused on continuing to expand our AI-powered spatial data platform worldwide. We have a significant presence in North America, Europe and Asia, with leadership teams and a go-to-market infrastructure in each of these regions. We have offices in London, Singapore and several across the United States, and we are accelerating our international expansion. As of December 31, 2022, we had over 701,000 subscribers in more than 170 countries. We believe that the geography-agnostic nature of our spatial data platform is a significant advantage as we continue to grow internationally.
•Broad patent portfolio supporting 10 years of R&D and innovation. As of December 31, 2022, we had 54 issued and 37 pending patent applications. Our success is based on almost 10 years of focus on innovation. Innovation has been at the center of Matterport, and we will continue to prioritize our investments in R&D to further our market leading position.
•Superior capture technology. Matterport’s capture technology platform is a software framework that enables support for a wide variety of capture devices required to create a Matterport digital twin of a building or space.
This includes support for LiDAR cameras, 360 cameras, smartphones, Matterport Axis and the Matterport Pro2 and Pro3 cameras. The Pro2 camera was foundational to our spatial data advantage, and we have expanded that advantage with an array of Matterport-enabled third-party capture devices. In August 2022, we launched and began shipment of our Pro3 Camera along with major updates to our industry-leading digital twin cloud platform. The Matterport Pro3 Camera is an advanced 3D capture device, which includes faster boot time, swappable batteries, and a lighter design. The Pro3 camera can perform both indoors and outdoors and is designed for speed, fidelity, versatility and accuracy. Along with our Pro2 Camera, we expect that future sales of our Pro3 Camera will continue to drive increased adoption of our solutions. Matterport is democratizing the 3D capture experience, making high-fidelity and high-accuracy 3D digital twins readily available for any building type and any subscriber need in the property life cycle. While there are other 3D capture solution providers, very few can produce true, dimensionally accurate 3D results, and fewer still can automatically create a final product in photorealistic 3D, and at global scale. This expansive capture technology offering would not be possible without our rich spatial data library available to train the AI-powered Cortex engine to automatically generate accurate digital twins from photos captured with a smartphone or 360 camera.
</document>"""

# Print GigaChat's response
print(get_completion(PROMPT))

<answer>701,000</answer>


How do we fix this? Well, a great way to reduce hallucinations on long documents is to **make GigaChat gather evidence first.** 

In this case, we **tell GigaChat to first extract relevant quotes, then base its answer on those quotes**. Telling GigaChat to do so here makes it correctly notice that the quote does not answer the question.

In [33]:
# Prompt
PROMPT = """<question>What was Matterport's subscriber base on the precise date of May 31, 2020?</question>
Please read the below document. Then, in <scratchpad> tags, pull the most relevant quote from the document and consider whether it answers the user's question or whether it lacks sufficient detail. Then write a brief numerical answer in <answer> tags.

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterport is leading the digitization and datafication of the built world. We believe the digital transformation of the built world will fundamentally change the way people interact with buildings and the physical spaces around them.
Since its founding in 2011, Matterport’s pioneering technology has set the standard for digitizing, accessing and managing buildings, spaces and places online. Our platform’s innovative software, spatial data-driven data science, and 3D capture technology have broken down the barriers that have kept the largest asset class in the world, buildings and physical spaces, offline and underutilized for many years. We believe the digitization and datafication of the built world will continue to unlock significant operational efficiencies and property values, and that Matterport is the platform to lead this enormous global transformation.
The world is rapidly moving from offline to online. Digital transformation has made a powerful and lasting impact across every business and industry today. According to International Data Corporation, or IDC, over $6.8 trillion of direct investments will be made on digital transformation from 2020 to 2023, the global digital transformation spending is forecasted to reach $3.4 trillion in 2026 with a five-year compound annual growth rate (“CAGR”) of 16.3%, and digital twin investments are expected to have a five-year CAGR of 35.2%. With this secular shift, there is also growing demand for the built world to transition from physical to digital. Nevertheless, the vast majority of buildings and spaces remain offline and undigitized. The global building stock, estimated by Savills to be $327 trillion in total property value as of 2021, remains largely offline today, and we estimate that less than 0.1% is penetrated by digital transformation.
Matterport was among the first to recognize the increasing need for digitization of the built world and the power of spatial data, the unique details underlying buildings and spaces, in facilitating the understanding of buildings and spaces. In the past, technology advanced physical road maps to the data-rich, digital maps and location services we all rely on today. Matterport now digitizes buildings, creating a data-rich environment to vastly increase our understanding and the full potential of each and every space we capture. Just as we can instantly, at the touch of a button, learn the fastest route from one city to another or locate the nearest coffee shops, Matterport’s spatial data for buildings unlocks a rich set of insights and learnings about properties and spaces worldwide. In addition, just as the geo-spatial mapping platforms of today have opened their mapping data to industry to create new business models such as ridesharing, e-commerce, food delivery marketplaces, and even short-term rental and home sharing, open access to Matterport’s structured spatial data is enabling new opportunities and business models for hospitality, facilities management, insurance, construction, real estate and retail, among others.
We believe the total addressable market opportunity for digitizing the built world is over $240 billion, and could be as high as $1 trillion as the market matures at scale. This is based on our analysis, modeling and understanding of the global building stock of over 4 billion properties and 20 billion spaces in the world today. With the help of artificial intelligence (“AI”), machine learning (“ML”) and deep learning (“DL”) technologies, we believe that, with the additional monetization opportunities from powerful spatial data-driven property insights and analytics, the total addressable market for the digitization and datafication of the built world will reach more than $1 trillion.

Our spatial data platform and capture of digital twins deliver value across a diverse set of industries and use cases. Large retailers can manage thousands of store locations remotely, real estate agencies can provide virtual open houses for hundreds of properties and thousands of visitors at the same time, property developers can monitor the entirety of the construction process with greater detail and speed, and insurance companies can more precisely document and evaluate claims and underwriting assessments with efficiency and precision. Matterport delivers the critical digital experience, tools and information that matter to our subscribers about properties of virtually any size, shape, and location worldwide.
For nearly a decade, we have been growing our spatial data platform and expanding our capabilities in order to create the most detailed, accurate, and data-rich digital twins available. Moreover, our 3D reconstruction process is fully automated, allowing our solution to scale with equal precision to millions of buildings and spaces of any type, shape, and size in the world. The universal applicability of our service provides Matterport significant scale and reach across diverse verticals and any geography. As of December 31, 2022, our subscriber base had grown approximately 39% to over 701,000 subscribers from 503,000 subscribers as of December 31, 2021, with our digital twins reaching more than 170 countries. We have digitized more than 28 billion square feet of space across multiple industries, representing significant scale and growth over the rest of the market.

As we continue to transform buildings into data worldwide, we are extending our spatial data platform to further transform property planning, development, management and intelligence for our subscribers across industries to become the de facto building and business intelligence engine for the built world. We believe the demand for spatial data and resulting insights for enterprises, businesses and institutions across industries, including real estate, architecture, engineering and construction (“AEC”), retail, insurance and government, will continue to grow rapidly.
We believe digitization and datafication represent a tremendous greenfield opportunity for growth across this massive category and asset class. From the early stages of design and development to marketing, operations, insurance and building repair and maintenance, our platform’s software and technology provide subscribers critical tools and insights to drive cost savings, increase revenues and optimally manage their buildings and spaces. We believe that hundreds of billions of dollars in unrealized utilization and operating efficiencies in the built world can be unlocked through the power of our spatial data platform. Our platform and data solutions have universal applicability across industries and building categories, giving Matterport a significant advantage as we can address the entirety of this large market opportunity and increase the value of what we believe to be the largest asset class in the world.
With a demonstrated track record of delivering value to our subscribers, our offerings include software subscription, data licensing, services and product hardware. As of December 31, 2022, our subscriber base included over 24% of Fortune 1000 companies, with less than 10% of our total revenue generated from our top 10 subscribers. We expect more than 80% of our revenue to come from our software subscription and data license solutions by 2025. Our innovative 3D capture products, the Pro2 and Pro3 Cameras, have played an integral part in shaping the 3D building and property visualization ecosystem. The Pro2 and Pro3 Cameras have driven adoption of our solutions and have generated the unique high-quality and scaled data set that has enabled Cortex, our proprietary AI software engine, to become the pioneering engine for digital twin creation. With this data advantage initially spurred by the Pro2 Camera, we have developed a capture device agnostic platform that scales and can generate new building and property insights for our subscribers across industries and geographies.
We have recently experienced rapid growth. Our subscribers have grown approximately 49-fold from December 31, 2018 to December 31, 2022. Our revenue increased by approximately 22% to $136.1 million for the year ended December 31, 2022, from approximately $111.2 million for the year ended December 31, 2021. Our gross profit decreased by $8.1 million or 14%, to $51.8 million for the year ended December 31, 2022, from $60.0 million for the year ended December 31, 2021, primarily attributable to certain disruptive and incremental costs due to the global supply chain constraints in fiscal year 2022. Our ability to retain and grow the subscription revenue generated by our existing subscribers is an important measure of the health of our business and our future growth prospects. We track our performance in this area by measuring our net dollar expansion rate from the same set of customers across comparable periods. Our net dollar expansion rate of 103% for the three months ended December 31, 2022 demonstrates the stickiness and growth potential of our platform.
Our Industry and Market Opportunity
Today, the vast majority of buildings and spaces remain undigitized. We estimate our current serviceable addressable market includes approximately 1.3 billion spaces worldwide, primarily from the real estate and travel and hospitality sectors. With approximately 9.2 million spaces under management as of December 31, 2022, we are continuing to penetrate the global building stock and expand our footprint across various end markets, including residential and commercial real estate, facilities management, retail, AEC, insurance and repair, and travel and hospitality. We estimate our total addressable market to be more than 4 billion buildings and 20 billion spaces globally, yielding a more than $240 billion market opportunity. We believe that as Matterport’s unique spatial data library and property data services continue to grow, this opportunity could increase to more than $1 trillion based on the size of the building stock and the untapped value creation available to buildings worldwide. The constraints created by the COVID-19 pandemic have only reinforced and accelerated the importance of our scaled 3D capture solution that we have developed for diverse industries and markets over the past decade.

Our Spatial Data Platform
Overview
Our technology platform uses spatial data collected from a wide variety of digital capture devices to transform physical buildings and spaces into dimensionally accurate, photorealistic digital twins that provide our subscribers access to previously unavailable building information and insights.
As a first mover in this massive market for nearly a decade, we have developed and scaled our industry-leading 3D reconstruction technology powered by Cortex, our proprietary AI-driven software engine that uses machine learning to recreate a photorealistic, 3D virtual representation of an entire building structure, including contents, equipment and furnishings. The finished product is a detailed and dynamic replication of the physical space that can be explored, analyzed and customized from a web browser on any device, including smartphones. The power to manage even large-scale commercial buildings is in the palm of each subscriber’s hands, made possible by our advanced technology and breakthrough innovations across our entire spatial data technology stack.
Key elements of our spatial data platform include:
•Bringing offline buildings online. Traditionally, our customers needed to conduct in-person site visits to understand and assess their buildings and spaces. While photographs and floor plans can be helpful, these forms of two-dimensional (“2D”) representation have limited information and tend to be static and rigid, and thus lack the interactive element critical to a holistic understanding of each building and space. With the AI-powered capabilities of Cortex, our proprietary AI software, representation of physical objects is no longer confined to static 2D images and physical visits can be eliminated. Cortex helps to move the buildings and spaces from offline to online and makes them accessible to our customers in real-time and on demand from anywhere. After subscribers scan their buildings, our visualization algorithms accurately infer spatial positions and depths from flat, 2D imagery captured through the scans and transform them into high- fidelity and precise digital twin models. This creates a fully automated image processing pipeline to ensure that each digital twin is of professional grade image quality.
•Driven by spatial data. We are a data-driven company. Each incremental capture of a space grows the richness and depth of our spatial data library. Spatial data represents the unique and idiosyncratic details that underlie and compose the buildings and spaces in the human- made environment. Cortex uses the breadth of the billions of data points we have accumulated over the years to improve the 3D accuracy of our digital twins. We help our subscribers pinpoint the height, location and other characteristics of objects in their digital twin. Our sophisticated algorithms also deliver significant commercial value to our subscribers by generating data-based insights that allow them to confidently make assessments and decisions about their properties. For instance, property developers can assess the amount of natural heat and daylight coming from specific windows, retailers can ensure each store layout is up to the same level of code and brand requirements, and factories can insure machinery layouts meet specifications and location guidelines. With approximately 9.2 million spaces under management as of December 31, 2022, our spatial data library is the clearinghouse for information about the built world.
•Powered by AI and ML. Artificial intelligence and machine learning technologies effectively utilize spatial data to create a robust virtual experience that is dynamic, realistic, interactive, informative and permits multiple viewing angles. AI and ML also make costly cameras unnecessary for everyday scans—subscribers can now scan their spaces by simply tapping a button on their smartphones. As a result, Matterport is a device agnostic platform, helping us more rapidly scale and drive towards our mission of digitizing and indexing the built world.
Our value proposition to subscribers is designed to serve the entirety of the digital building lifecycle, from design and build to maintenance and operations, promotion, sale, lease, insure, repair, restore, secure and finance. As a result, we believe we are uniquely positioned to grow our revenue with our subscribers as we help them to discover opportunities to drive short- and long-term return on investment by taking their buildings and spaces from offline to online across their portfolios of properties.
Ubiquitous Capture
Matterport has become the standard for 3D space capture. Our technology platform empowers subscribers worldwide to quickly, easily and accurately digitize, customize and manage interactive and dimensionally accurate digital twins of their buildings and spaces.
The Matterport platform is designed to work with a wide range of LiDAR, spherical, 3D and 360 cameras, as well as smartphones, to suit the capture needs of all of our subscribers. This provides the flexibility to capture a space of any size, scale, and complexity, at anytime and anywhere.
•Matterport Pro3 is our newest 3D camera that scans properties faster than earlier versions to help accelerate project completion. Pro3 provides the highest accuracy scans of both indoor and outdoor spaces and is designed for speed, fidelity, versatility and accuracy. Capturing 3D data up to 100 meters away at less than 20 seconds per sweep, Pro3’s ultra-fast, high-precision LiDAR sensor can run for hours and takes millions of measurements in any conditions.
•Matterport Pro2 is our proprietary 3D camera that has been used to capture millions of spaces around the world with a high degree of fidelity, precision, speed and simplicity. Capable of capturing buildings more than 500,000 square feet in size, it has become the camera of choice for many residential, commercial, industrial and large-scale properties.
•360 Cameras. Matterport supports a selection of 360 cameras available in the market. These affordable, pocket sized devices deliver precision captures with high fidelity and are appropriate for capturing smaller homes, condos, short-term rentals, apartments, and more. The spherical lens image capture technology of these devices gives Cortex robust, detailed image data to transform panoramas into our industry-leading digital twins.
•LEICA BLK360. Through our partnership with Leica, our 3D reconstruction technology and our AI powered software engine, Cortex, transform this powerful LiDAR camera into an ultra-precise capture device for creating Matterport digital twins. It is the solution of choice for AEC professionals when exacting precision is required.
•Smartphone Capture. Our capture apps are commercially available for both iOS and Android. Matterport’s smartphone capture solution has democratized 3D capture, making it easy and accessible for anyone to digitize buildings and spaces with a recent iPhone device since the initial introduction of Matterport for iPhone in May 2020. In April 2021, we announced the official release of the Android Capture app, giving Android users the ability to quickly and easily capture buildings and spaces in immersive 3D. In February 2022, we launched Matterport Axis, a motorized mount that holds a smartphone and can be used with the Matterport Capture app to capture 3D digital twins of any physical space with increased speed, precision, and consistency.
Cortex and 3D Reconstruction (the Matterport Digital Twin)
With a spatial data library, as of December 31, 2022, of approximately 9.2 million spaces under management, representing approximately 28 billion captured square feet of space, we use our advanced ML and DL technologies to algorithmically transform the spatial data we capture into an accurate 3D digital reproduction of any physical space. This intelligent, automated 3D reconstruction is made possible by Cortex, our AI-powered software engine that includes a deep learning neural network that uses our spatial data library to understand how a building or space is divided into floors and rooms, where the doorways and openings are located, and what types of rooms are present, such that those forms are compiled and aligned with dimensional accuracy into a dynamic, photorealistic digital twin. Other components of Cortex include AI-powered computer vision technologies to identify and classify the contents inside a building or space, and object recognition technologies to identify and segment everything from furnishings and equipment to doors, windows, light fixtures, fire suppression sprinklers and fire escapes. Our highly scalable artificial intelligence platform enables our subscribers to tap into powerful, enhanced building data and insights at the click of a button.

The Science Behind the Matterport Digital Twin: Cortex AI Highlights
Matterport Runs on Cortex
Cortex is our AI-powered software engine that includes a precision deep learning neural network to create digital twins of any building or space. Developed using our proprietary spatial data captured with our Pro2 and Pro3 cameras, Cortex delivers a high degree of precision and accuracy while enabling 3D capture using everyday devices.
Generic neural networks struggle with 3D reconstruction of the real world. Matterport-optimized networks deliver more accurate and robust results. More than just raw training data, Matterport’s datasets allow us to develop new neural network architectures and evaluate them against user behavior and real-world data in millions of situations.
•Deep learning: Connecting and optimizing the detailed neural network data architecture of each space is key to creating robust, highly accurate 3D digital twins. Cortex evaluates and optimizes each 3D model against Matterport’s rich spatial data aggregated from millions of buildings and spaces and the human annotations of those data provided by tens of thousands of subscribers worldwide. Cortex’s evaluative abilities and its data-driven optimization of 3D reconstruction yield consistent, high-precision results across a wide array of building configurations, spaces and environments.
•Dynamic 3D reconstruction: Creating precise 3D spatial data at scale from 2D visuals and static images requires a combination of photorealistic, detailed data from multiple viewpoints and millions of spaces that train and optimize Cortex’s neural network and learning capabilities for improved 3D reconstruction of any space. Cortex’s capabilities combined with real-time spatial alignment algorithms in our 3D capture technology create an intuitive “preview” of any work in progress, allowing subscribers to work with their content interactively and in real-time.
•Computer vision: Cortex enables a suite of powerful features to enhance the value of digital twins. These include automatic measurements for rooms or objects in a room, automatic 2D-from-3D high-definition photo gallery creation, auto face blurring for privacy protection, custom videos, walkthroughs, auto room labeling and object recognition.
•Advanced image processing: Matterport’s computational photography algorithms create a fully automated image processing pipeline to help ensure that each digital twin is of professional grade image quality. Our patented technology makes 3D capture as simple as pressing a single button. Matterport’s software and technology manage the remaining steps, including white balance and camera-specific color correction, high dynamic range tone mapping, de-noising, haze removal, sharpening, saturation and other adjustments to improve image quality.
Spatial Data and AI-Powered Insights
Every Matterport digital twin contains extensive information about a building, room or physical space. The data uses our AI-powered Cortex engine. In addition to the Matterport digital twin itself, our spatial data consists of precision building geometry and structural detail, building contents, fixtures and condition, along with high-definition imagery and photorealistic detail from many vantage points in a space. Cortex employs a technique we call deep spatial indexing. Deep spatial indexing uses artificial intelligence, computer vision and deep learning to identify and convey important details about each space, its structure and its contents with precision and fidelity. We have created a robust spatial data standard that enables Matterport subscribers to harness an interoperable digital system of record for any building.
In addition to creating a highly interactive digital experience for subscribers through the construction of digital twins, we ask ourselves two questions for every subscriber: (1) what is important about their building or physical space and (2) what learnings and insights can we deliver for this space? Our AI-powered Cortex engine helps us answer these questions using our spatial data library to provide aggregated property trends and operational and valuation insights. Moreover, as the Matterport platform ecosystem continues to expand, our subscribers, partners and other third-party developers can bring their own tools to further the breadth and depth of insights they can harvest from our rich spatial data layer.
Extensible Platform Ecosystem
Matterport offers the largest and most accurate library of spatial data in the world, with, as of December 31, 2022, approximately 9.2 million spaces under management and approximately 28 billion captured square feet. The versatility of our spatial data platform and extensive enterprise software development kit and application programming interfaces (“APIs”) has allowed us to develop a robust global ecosystem of channels and partners that extend the Matterport value proposition by geography and vertical market. We intend to continue to deploy a broad set of workflow integrations with our partners and their subscribers to promote an integrated Matterport solution across our target markets. We are also developing a third-party software marketplace to extend the power of our spatial data platform with easy-to-deploy and easy-to-access Matterport software add-ons. The marketplace enables developers to build new applications and spatial data mining tools, enhance the Matterport 3D experience, and create new productivity and property management tools that supplement our core offerings. These value-added capabilities created by third-party developers enable a scalable new revenue stream, with Matterport sharing the subscription and services revenue from each add-on that is deployed to subscribers through the online marketplace. The network effects of our platform ecosystem contributes to the growth of our business, and we believe that it will continue to bolster future growth by enhancing subscriber stickiness and user engagement.
Examples of Matterport add-ons and extensions include:
•Add-ons: Encircle (easy-to-use field documentation tools for faster claims processing); WP Matterport Shortcode (free Wordpress plugin that allows Matterport to be embedded quickly and easily with a Matterport shortcode), WP3D Models (WordPress + Matterport integration plugin); Rela (all-in-one marketing solution for listings); CAPTUR3D (all-in-one Content Management System that extends value to Matterport digital twins); Private Model Emded (feature that allows enterprises to privately share digital twins with a large group of employees on the corporate network without requiring additional user licenses); Views (new workgroup collaboration framework to enable groups and large organizations to create separate, permissions-based workflows to manage different tasks with different teams); and Guided Tours and Tags (tool to elevate the visitor experience by creating directed virtual tours of any commercial or residential space tailored to the interests of their visitors). We unveiled our private beta integration with Amazon Web Services (AWS) IoT TwinMaker to enable enterprise customers to seamlessly connect IoT data into visually immersive and dimensionally accurate Matterport digital twin.
•Services: Matterport ADA Compliant Digital Twin (solution to provide American Disability Act compliant digital twins) and Enterprise Cloud Software Platform (reimagined cloud software platform for the enterprise that creates, publishes, and manages digital twins of buildings and spaces of any size of shape, indoors or outdoors).
Our Competitive Strengths
We believe that we have a number of competitive strengths that will enable our market leadership to grow. Our competitive strengths include:
•Breadth and depth of the Matterport platform. Our core strength is our all-in-one spatial data platform with broad reach across diverse verticals and geographies such as capture to processing to industries without customization. With the ability to integrate seamlessly with various enterprise systems, our platform delivers value across the property lifecycle for diverse end markets, including real estate, AEC, travel and hospitality, repair and insurance, and industrial and facilities. As of December 31, 2022, our global reach extended to subscribers in more than 170 countries, including over 24% of Fortune 1000 companies.
•Market leadership and first-mover advantage. Matterport defined the category of digitizing and datafying the built world almost a decade ago, and we have become the global leader in the category. As of December 31, 2022, we had over 701,000 subscribers on our platform and approximately 9.2 million spaces under management. Our leadership is primarily driven by the fact that we were the first mover in digital twin creation. As a result of our first mover advantage, we have amassed a deep and rich library of spatial data that continues to compound and enhance our leadership position.
•Significant network effect. With each new capture and piece of data added to our platform, the richness of our dataset and the depth of insights from our spaces under management grow. In addition, the combination of our ability to turn data into insights with incremental data from new data captures by our subscribers enables Matterport to develop features for subscribers to our platform. We were a first mover in building a spatial data library for the built world, and our leadership in gathering and deriving insights from data continues to compound and the relevance of those insights attracts more new subscribers.
•Massive spatial data library as the raw material for valuable property insights. The scale of our spatial data library is a significant advantage in deriving insights for our subscribers. Our spatial data library serves as vital ground truth for Cortex, enabling Matterport to create powerful 3D digital twins using a wide range of camera technology, including low-cost digital and smartphone cameras. As of December 31, 2022, our data came from approximately 9.2 million spaces under management and approximately 28 billion captured square feet. As a result, we have taken property insights and analytics to new levels, benefiting subscribers across various industries. For example, facilities managers significantly reduce the time needed to create building layouts, leading to a significant decrease in the cost of site surveying and as-built modeling. AEC subscribers use the analytics of each as-built space to streamline documentation and collaborate with ease.
•Global reach and scale. We are focused on continuing to expand our AI-powered spatial data platform worldwide. We have a significant presence in North America, Europe and Asia, with leadership teams and a go-to-market infrastructure in each of these regions. We have offices in London, Singapore and several across the United States, and we are accelerating our international expansion. As of December 31, 2022, we had over 701,000 subscribers in more than 170 countries. We believe that the geography-agnostic nature of our spatial data platform is a significant advantage as we continue to grow internationally.
•Broad patent portfolio supporting 10 years of R&D and innovation. As of December 31, 2022, we had 54 issued and 37 pending patent applications. Our success is based on almost 10 years of focus on innovation. Innovation has been at the center of Matterport, and we will continue to prioritize our investments in R&D to further our market leading position.
•Superior capture technology. Matterport’s capture technology platform is a software framework that enables support for a wide variety of capture devices required to create a Matterport digital twin of a building or space.
This includes support for LiDAR cameras, 360 cameras, smartphones, Matterport Axis and the Matterport Pro2 and Pro3 cameras. The Pro2 camera was foundational to our spatial data advantage, and we have expanded that advantage with an array of Matterport-enabled third-party capture devices. In August 2022, we launched and began shipment of our Pro3 Camera along with major updates to our industry-leading digital twin cloud platform. The Matterport Pro3 Camera is an advanced 3D capture device, which includes faster boot time, swappable batteries, and a lighter design. The Pro3 camera can perform both indoors and outdoors and is designed for speed, fidelity, versatility and accuracy. Along with our Pro2 Camera, we expect that future sales of our Pro3 Camera will continue to drive increased adoption of our solutions. Matterport is democratizing the 3D capture experience, making high-fidelity and high-accuracy 3D digital twins readily available for any building type and any subscriber need in the property life cycle. While there are other 3D capture solution providers, very few can produce true, dimensionally accurate 3D results, and fewer still can automatically create a final product in photorealistic 3D, and at global scale. This expansive capture technology offering would not be possible without our rich spatial data library available to train the AI-powered Cortex engine to automatically generate accurate digital twins from photos captured with a smartphone or 360 camera.
</document>"""

# Print GigaChat's response
print(get_completion(PROMPT))

**<scratchpad>**

Relevant Quote:
> "As of December 31, 2022, our subscriber base had grown approximately 39% to over 701,000 subscribers from 503,000 subscribers as of December 31, 2021."

However, the document does **not** specify the precise subscriber base on May 31, 2020.

**<answer>**
0 (no data available for May 31, 2020)


---

## Exercises
- [Exercise 8.1 - Beyoncé Hallucination](#exercise-81---beyoncé-hallucination)
- [Exercise 8.2 - Prospectus Hallucination](#exercise-82---prospectus-hallucination)

### Exercise 8.1 - Beyoncé Hallucination
Modify the `PROMPT` to fix GigaChat's hallucination issue by giving GigaChat an out. (Renaissance is Beyoncé's seventh studio album, not her eigthth.)

It is recommended to run the cell first to observe hallucinations before attempting fixes.

In [ ]:
# Prompt
PROMPT = "In what year did star performer Beyoncé release her eighth studio album?"

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    contains = bool(
        re.search("Unfortunately", text) or
        re.search("I do not", text) or
        re.search("I don't", text)
    )
    does_not_contain = not bool(re.search("2022", text))
    return contains and does_not_contain

# Print GigaChat's response and the corresponding grade
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_8_1_hint; print(exercise_8_1_hint)

### Exercise 8.1 - Prospectus Hallucination
Modify the `PROMPT` to fix GigaChat's hallucination issue by asking for citations. The correct answer is that subscribers went up 49x.

In [ ]:
# Prompt
PROMPT = """From December 2018 to December 2022, by what amount did Matterport's subscribers grow?

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterport is leading the digitization and datafication of the built world. We believe the digital transformation of the built world will fundamentally change the way people interact with buildings and the physical spaces around them.
Since its founding in 2011, Matterport’s pioneering technology has set the standard for digitizing, accessing and managing buildings, spaces and places online. Our platform’s innovative software, spatial data-driven data science, and 3D capture technology have broken down the barriers that have kept the largest asset class in the world, buildings and physical spaces, offline and underutilized for many years. We believe the digitization and datafication of the built world will continue to unlock significant operational efficiencies and property values, and that Matterport is the platform to lead this enormous global transformation.
The world is rapidly moving from offline to online. Digital transformation has made a powerful and lasting impact across every business and industry today. According to International Data Corporation, or IDC, over $6.8 trillion of direct investments will be made on digital transformation from 2020 to 2023, the global digital transformation spending is forecasted to reach $3.4 trillion in 2026 with a five-year compound annual growth rate (“CAGR”) of 16.3%, and digital twin investments are expected to have a five-year CAGR of 35.2%. With this secular shift, there is also growing demand for the built world to transition from physical to digital. Nevertheless, the vast majority of buildings and spaces remain offline and undigitized. The global building stock, estimated by Savills to be $327 trillion in total property value as of 2021, remains largely offline today, and we estimate that less than 0.1% is penetrated by digital transformation.
Matterport was among the first to recognize the increasing need for digitization of the built world and the power of spatial data, the unique details underlying buildings and spaces, in facilitating the understanding of buildings and spaces. In the past, technology advanced physical road maps to the data-rich, digital maps and location services we all rely on today. Matterport now digitizes buildings, creating a data-rich environment to vastly increase our understanding and the full potential of each and every space we capture. Just as we can instantly, at the touch of a button, learn the fastest route from one city to another or locate the nearest coffee shops, Matterport’s spatial data for buildings unlocks a rich set of insights and learnings about properties and spaces worldwide. In addition, just as the geo-spatial mapping platforms of today have opened their mapping data to industry to create new business models such as ridesharing, e-commerce, food delivery marketplaces, and even short-term rental and home sharing, open access to Matterport’s structured spatial data is enabling new opportunities and business models for hospitality, facilities management, insurance, construction, real estate and retail, among others.
We believe the total addressable market opportunity for digitizing the built world is over $240 billion, and could be as high as $1 trillion as the market matures at scale. This is based on our analysis, modeling and understanding of the global building stock of over 4 billion properties and 20 billion spaces in the world today. With the help of artificial intelligence (“AI”), machine learning (“ML”) and deep learning (“DL”) technologies, we believe that, with the additional monetization opportunities from powerful spatial data-driven property insights and analytics, the total addressable market for the digitization and datafication of the built world will reach more than $1 trillion.

Our spatial data platform and capture of digital twins deliver value across a diverse set of industries and use cases. Large retailers can manage thousands of store locations remotely, real estate agencies can provide virtual open houses for hundreds of properties and thousands of visitors at the same time, property developers can monitor the entirety of the construction process with greater detail and speed, and insurance companies can more precisely document and evaluate claims and underwriting assessments with efficiency and precision. Matterport delivers the critical digital experience, tools and information that matter to our subscribers about properties of virtually any size, shape, and location worldwide.
For nearly a decade, we have been growing our spatial data platform and expanding our capabilities in order to create the most detailed, accurate, and data-rich digital twins available. Moreover, our 3D reconstruction process is fully automated, allowing our solution to scale with equal precision to millions of buildings and spaces of any type, shape, and size in the world. The universal applicability of our service provides Matterport significant scale and reach across diverse verticals and any geography. As of December 31, 2022, our subscriber base had grown approximately 39% to over 701,000 subscribers from 503,000 subscribers as of December 31, 2021, with our digital twins reaching more than 170 countries. We have digitized more than 28 billion square feet of space across multiple industries, representing significant scale and growth over the rest of the market.

As we continue to transform buildings into data worldwide, we are extending our spatial data platform to further transform property planning, development, management and intelligence for our subscribers across industries to become the de facto building and business intelligence engine for the built world. We believe the demand for spatial data and resulting insights for enterprises, businesses and institutions across industries, including real estate, architecture, engineering and construction (“AEC”), retail, insurance and government, will continue to grow rapidly.
We believe digitization and datafication represent a tremendous greenfield opportunity for growth across this massive category and asset class. From the early stages of design and development to marketing, operations, insurance and building repair and maintenance, our platform’s software and technology provide subscribers critical tools and insights to drive cost savings, increase revenues and optimally manage their buildings and spaces. We believe that hundreds of billions of dollars in unrealized utilization and operating efficiencies in the built world can be unlocked through the power of our spatial data platform. Our platform and data solutions have universal applicability across industries and building categories, giving Matterport a significant advantage as we can address the entirety of this large market opportunity and increase the value of what we believe to be the largest asset class in the world.
With a demonstrated track record of delivering value to our subscribers, our offerings include software subscription, data licensing, services and product hardware. As of December 31, 2022, our subscriber base included over 24% of Fortune 1000 companies, with less than 10% of our total revenue generated from our top 10 subscribers. We expect more than 80% of our revenue to come from our software subscription and data license solutions by 2025. Our innovative 3D capture products, the Pro2 and Pro3 Cameras, have played an integral part in shaping the 3D building and property visualization ecosystem. The Pro2 and Pro3 Cameras have driven adoption of our solutions and have generated the unique high-quality and scaled data set that has enabled Cortex, our proprietary AI software engine, to become the pioneering engine for digital twin creation. With this data advantage initially spurred by the Pro2 Camera, we have developed a capture device agnostic platform that scales and can generate new building and property insights for our subscribers across industries and geographies.
We have recently experienced rapid growth. Our subscribers have grown approximately 49-fold from December 31, 2018 to December 31, 2022. Our revenue increased by approximately 22% to $136.1 million for the year ended December 31, 2022, from approximately $111.2 million for the year ended December 31, 2021. Our gross profit decreased by $8.1 million or 14%, to $51.8 million for the year ended December 31, 2022, from $60.0 million for the year ended December 31, 2021, primarily attributable to certain disruptive and incremental costs due to the global supply chain constraints in fiscal year 2022. Our ability to retain and grow the subscription revenue generated by our existing subscribers is an important measure of the health of our business and our future growth prospects. We track our performance in this area by measuring our net dollar expansion rate from the same set of customers across comparable periods. Our net dollar expansion rate of 103% for the three months ended December 31, 2022 demonstrates the stickiness and growth potential of our platform.
Our Industry and Market Opportunity
Today, the vast majority of buildings and spaces remain undigitized. We estimate our current serviceable addressable market includes approximately 1.3 billion spaces worldwide, primarily from the real estate and travel and hospitality sectors. With approximately 9.2 million spaces under management as of December 31, 2022, we are continuing to penetrate the global building stock and expand our footprint across various end markets, including residential and commercial real estate, facilities management, retail, AEC, insurance and repair, and travel and hospitality. We estimate our total addressable market to be more than 4 billion buildings and 20 billion spaces globally, yielding a more than $240 billion market opportunity. We believe that as Matterport’s unique spatial data library and property data services continue to grow, this opportunity could increase to more than $1 trillion based on the size of the building stock and the untapped value creation available to buildings worldwide. The constraints created by the COVID-19 pandemic have only reinforced and accelerated the importance of our scaled 3D capture solution that we have developed for diverse industries and markets over the past decade.

Our Spatial Data Platform
Overview
Our technology platform uses spatial data collected from a wide variety of digital capture devices to transform physical buildings and spaces into dimensionally accurate, photorealistic digital twins that provide our subscribers access to previously unavailable building information and insights.
As a first mover in this massive market for nearly a decade, we have developed and scaled our industry-leading 3D reconstruction technology powered by Cortex, our proprietary AI-driven software engine that uses machine learning to recreate a photorealistic, 3D virtual representation of an entire building structure, including contents, equipment and furnishings. The finished product is a detailed and dynamic replication of the physical space that can be explored, analyzed and customized from a web browser on any device, including smartphones. The power to manage even large-scale commercial buildings is in the palm of each subscriber’s hands, made possible by our advanced technology and breakthrough innovations across our entire spatial data technology stack.
Key elements of our spatial data platform include:
•Bringing offline buildings online. Traditionally, our customers needed to conduct in-person site visits to understand and assess their buildings and spaces. While photographs and floor plans can be helpful, these forms of two-dimensional (“2D”) representation have limited information and tend to be static and rigid, and thus lack the interactive element critical to a holistic understanding of each building and space. With the AI-powered capabilities of Cortex, our proprietary AI software, representation of physical objects is no longer confined to static 2D images and physical visits can be eliminated. Cortex helps to move the buildings and spaces from offline to online and makes them accessible to our customers in real-time and on demand from anywhere. After subscribers scan their buildings, our visualization algorithms accurately infer spatial positions and depths from flat, 2D imagery captured through the scans and transform them into high- fidelity and precise digital twin models. This creates a fully automated image processing pipeline to ensure that each digital twin is of professional grade image quality.
•Driven by spatial data. We are a data-driven company. Each incremental capture of a space grows the richness and depth of our spatial data library. Spatial data represents the unique and idiosyncratic details that underlie and compose the buildings and spaces in the human- made environment. Cortex uses the breadth of the billions of data points we have accumulated over the years to improve the 3D accuracy of our digital twins. We help our subscribers pinpoint the height, location and other characteristics of objects in their digital twin. Our sophisticated algorithms also deliver significant commercial value to our subscribers by generating data-based insights that allow them to confidently make assessments and decisions about their properties. For instance, property developers can assess the amount of natural heat and daylight coming from specific windows, retailers can ensure each store layout is up to the same level of code and brand requirements, and factories can insure machinery layouts meet specifications and location guidelines. With approximately 9.2 million spaces under management as of December 31, 2022, our spatial data library is the clearinghouse for information about the built world.
•Powered by AI and ML. Artificial intelligence and machine learning technologies effectively utilize spatial data to create a robust virtual experience that is dynamic, realistic, interactive, informative and permits multiple viewing angles. AI and ML also make costly cameras unnecessary for everyday scans—subscribers can now scan their spaces by simply tapping a button on their smartphones. As a result, Matterport is a device agnostic platform, helping us more rapidly scale and drive towards our mission of digitizing and indexing the built world.
Our value proposition to subscribers is designed to serve the entirety of the digital building lifecycle, from design and build to maintenance and operations, promotion, sale, lease, insure, repair, restore, secure and finance. As a result, we believe we are uniquely positioned to grow our revenue with our subscribers as we help them to discover opportunities to drive short- and long-term return on investment by taking their buildings and spaces from offline to online across their portfolios of properties.
Ubiquitous Capture
Matterport has become the standard for 3D space capture. Our technology platform empowers subscribers worldwide to quickly, easily and accurately digitize, customize and manage interactive and dimensionally accurate digital twins of their buildings and spaces.
The Matterport platform is designed to work with a wide range of LiDAR, spherical, 3D and 360 cameras, as well as smartphones, to suit the capture needs of all of our subscribers. This provides the flexibility to capture a space of any size, scale, and complexity, at anytime and anywhere.
•Matterport Pro3 is our newest 3D camera that scans properties faster than earlier versions to help accelerate project completion. Pro3 provides the highest accuracy scans of both indoor and outdoor spaces and is designed for speed, fidelity, versatility and accuracy. Capturing 3D data up to 100 meters away at less than 20 seconds per sweep, Pro3’s ultra-fast, high-precision LiDAR sensor can run for hours and takes millions of measurements in any conditions.
•Matterport Pro2 is our proprietary 3D camera that has been used to capture millions of spaces around the world with a high degree of fidelity, precision, speed and simplicity. Capable of capturing buildings more than 500,000 square feet in size, it has become the camera of choice for many residential, commercial, industrial and large-scale properties.
•360 Cameras. Matterport supports a selection of 360 cameras available in the market. These affordable, pocket sized devices deliver precision captures with high fidelity and are appropriate for capturing smaller homes, condos, short-term rentals, apartments, and more. The spherical lens image capture technology of these devices gives Cortex robust, detailed image data to transform panoramas into our industry-leading digital twins.
•LEICA BLK360. Through our partnership with Leica, our 3D reconstruction technology and our AI powered software engine, Cortex, transform this powerful LiDAR camera into an ultra-precise capture device for creating Matterport digital twins. It is the solution of choice for AEC professionals when exacting precision is required.
•Smartphone Capture. Our capture apps are commercially available for both iOS and Android. Matterport’s smartphone capture solution has democratized 3D capture, making it easy and accessible for anyone to digitize buildings and spaces with a recent iPhone device since the initial introduction of Matterport for iPhone in May 2020. In April 2021, we announced the official release of the Android Capture app, giving Android users the ability to quickly and easily capture buildings and spaces in immersive 3D. In February 2022, we launched Matterport Axis, a motorized mount that holds a smartphone and can be used with the Matterport Capture app to capture 3D digital twins of any physical space with increased speed, precision, and consistency.
Cortex and 3D Reconstruction (the Matterport Digital Twin)
With a spatial data library, as of December 31, 2022, of approximately 9.2 million spaces under management, representing approximately 28 billion captured square feet of space, we use our advanced ML and DL technologies to algorithmically transform the spatial data we capture into an accurate 3D digital reproduction of any physical space. This intelligent, automated 3D reconstruction is made possible by Cortex, our AI-powered software engine that includes a deep learning neural network that uses our spatial data library to understand how a building or space is divided into floors and rooms, where the doorways and openings are located, and what types of rooms are present, such that those forms are compiled and aligned with dimensional accuracy into a dynamic, photorealistic digital twin. Other components of Cortex include AI-powered computer vision technologies to identify and classify the contents inside a building or space, and object recognition technologies to identify and segment everything from furnishings and equipment to doors, windows, light fixtures, fire suppression sprinklers and fire escapes. Our highly scalable artificial intelligence platform enables our subscribers to tap into powerful, enhanced building data and insights at the click of a button.

The Science Behind the Matterport Digital Twin: Cortex AI Highlights
Matterport Runs on Cortex
Cortex is our AI-powered software engine that includes a precision deep learning neural network to create digital twins of any building or space. Developed using our proprietary spatial data captured with our Pro2 and Pro3 cameras, Cortex delivers a high degree of precision and accuracy while enabling 3D capture using everyday devices.
Generic neural networks struggle with 3D reconstruction of the real world. Matterport-optimized networks deliver more accurate and robust results. More than just raw training data, Matterport’s datasets allow us to develop new neural network architectures and evaluate them against user behavior and real-world data in millions of situations.
•Deep learning: Connecting and optimizing the detailed neural network data architecture of each space is key to creating robust, highly accurate 3D digital twins. Cortex evaluates and optimizes each 3D model against Matterport’s rich spatial data aggregated from millions of buildings and spaces and the human annotations of those data provided by tens of thousands of subscribers worldwide. Cortex’s evaluative abilities and its data-driven optimization of 3D reconstruction yield consistent, high-precision results across a wide array of building configurations, spaces and environments.
•Dynamic 3D reconstruction: Creating precise 3D spatial data at scale from 2D visuals and static images requires a combination of photorealistic, detailed data from multiple viewpoints and millions of spaces that train and optimize Cortex’s neural network and learning capabilities for improved 3D reconstruction of any space. Cortex’s capabilities combined with real-time spatial alignment algorithms in our 3D capture technology create an intuitive “preview” of any work in progress, allowing subscribers to work with their content interactively and in real-time.
•Computer vision: Cortex enables a suite of powerful features to enhance the value of digital twins. These include automatic measurements for rooms or objects in a room, automatic 2D-from-3D high-definition photo gallery creation, auto face blurring for privacy protection, custom videos, walkthroughs, auto room labeling and object recognition.
•Advanced image processing: Matterport’s computational photography algorithms create a fully automated image processing pipeline to help ensure that each digital twin is of professional grade image quality. Our patented technology makes 3D capture as simple as pressing a single button. Matterport’s software and technology manage the remaining steps, including white balance and camera-specific color correction, high dynamic range tone mapping, de-noising, haze removal, sharpening, saturation and other adjustments to improve image quality.
Spatial Data and AI-Powered Insights
Every Matterport digital twin contains extensive information about a building, room or physical space. The data uses our AI-powered Cortex engine. In addition to the Matterport digital twin itself, our spatial data consists of precision building geometry and structural detail, building contents, fixtures and condition, along with high-definition imagery and photorealistic detail from many vantage points in a space. Cortex employs a technique we call deep spatial indexing. Deep spatial indexing uses artificial intelligence, computer vision and deep learning to identify and convey important details about each space, its structure and its contents with precision and fidelity. We have created a robust spatial data standard that enables Matterport subscribers to harness an interoperable digital system of record for any building.
In addition to creating a highly interactive digital experience for subscribers through the construction of digital twins, we ask ourselves two questions for every subscriber: (1) what is important about their building or physical space and (2) what learnings and insights can we deliver for this space? Our AI-powered Cortex engine helps us answer these questions using our spatial data library to provide aggregated property trends and operational and valuation insights. Moreover, as the Matterport platform ecosystem continues to expand, our subscribers, partners and other third-party developers can bring their own tools to further the breadth and depth of insights they can harvest from our rich spatial data layer.
Extensible Platform Ecosystem
Matterport offers the largest and most accurate library of spatial data in the world, with, as of December 31, 2022, approximately 9.2 million spaces under management and approximately 28 billion captured square feet. The versatility of our spatial data platform and extensive enterprise software development kit and application programming interfaces (“APIs”) has allowed us to develop a robust global ecosystem of channels and partners that extend the Matterport value proposition by geography and vertical market. We intend to continue to deploy a broad set of workflow integrations with our partners and their subscribers to promote an integrated Matterport solution across our target markets. We are also developing a third-party software marketplace to extend the power of our spatial data platform with easy-to-deploy and easy-to-access Matterport software add-ons. The marketplace enables developers to build new applications and spatial data mining tools, enhance the Matterport 3D experience, and create new productivity and property management tools that supplement our core offerings. These value-added capabilities created by third-party developers enable a scalable new revenue stream, with Matterport sharing the subscription and services revenue from each add-on that is deployed to subscribers through the online marketplace. The network effects of our platform ecosystem contributes to the growth of our business, and we believe that it will continue to bolster future growth by enhancing subscriber stickiness and user engagement.
Examples of Matterport add-ons and extensions include:
•Add-ons: Encircle (easy-to-use field documentation tools for faster claims processing); WP Matterport Shortcode (free Wordpress plugin that allows Matterport to be embedded quickly and easily with a Matterport shortcode), WP3D Models (WordPress + Matterport integration plugin); Rela (all-in-one marketing solution for listings); CAPTUR3D (all-in-one Content Management System that extends value to Matterport digital twins); Private Model Emded (feature that allows enterprises to privately share digital twins with a large group of employees on the corporate network without requiring additional user licenses); Views (new workgroup collaboration framework to enable groups and large organizations to create separate, permissions-based workflows to manage different tasks with different teams); and Guided Tours and Tags (tool to elevate the visitor experience by creating directed virtual tours of any commercial or residential space tailored to the interests of their visitors). We unveiled our private beta integration with Amazon Web Services (AWS) IoT TwinMaker to enable enterprise customers to seamlessly connect IoT data into visually immersive and dimensionally accurate Matterport digital twin.
•Services: Matterport ADA Compliant Digital Twin (solution to provide American Disability Act compliant digital twins) and Enterprise Cloud Software Platform (reimagined cloud software platform for the enterprise that creates, publishes, and manages digital twins of buildings and spaces of any size of shape, indoors or outdoors).
Our Competitive Strengths
We believe that we have a number of competitive strengths that will enable our market leadership to grow. Our competitive strengths include:
•Breadth and depth of the Matterport platform. Our core strength is our all-in-one spatial data platform with broad reach across diverse verticals and geographies such as capture to processing to industries without customization. With the ability to integrate seamlessly with various enterprise systems, our platform delivers value across the property lifecycle for diverse end markets, including real estate, AEC, travel and hospitality, repair and insurance, and industrial and facilities. As of December 31, 2022, our global reach extended to subscribers in more than 170 countries, including over 24% of Fortune 1000 companies.
•Market leadership and first-mover advantage. Matterport defined the category of digitizing and datafying the built world almost a decade ago, and we have become the global leader in the category. As of December 31, 2022, we had over 701,000 subscribers on our platform and approximately 9.2 million spaces under management. Our leadership is primarily driven by the fact that we were the first mover in digital twin creation. As a result of our first mover advantage, we have amassed a deep and rich library of spatial data that continues to compound and enhance our leadership position.
•Significant network effect. With each new capture and piece of data added to our platform, the richness of our dataset and the depth of insights from our spaces under management grow. In addition, the combination of our ability to turn data into insights with incremental data from new data captures by our subscribers enables Matterport to develop features for subscribers to our platform. We were a first mover in building a spatial data library for the built world, and our leadership in gathering and deriving insights from data continues to compound and the relevance of those insights attracts more new subscribers.
•Massive spatial data library as the raw material for valuable property insights. The scale of our spatial data library is a significant advantage in deriving insights for our subscribers. Our spatial data library serves as vital ground truth for Cortex, enabling Matterport to create powerful 3D digital twins using a wide range of camera technology, including low-cost digital and smartphone cameras. As of December 31, 2022, our data came from approximately 9.2 million spaces under management and approximately 28 billion captured square feet. As a result, we have taken property insights and analytics to new levels, benefiting subscribers across various industries. For example, facilities managers significantly reduce the time needed to create building layouts, leading to a significant decrease in the cost of site surveying and as-built modeling. AEC subscribers use the analytics of each as-built space to streamline documentation and collaborate with ease.
•Global reach and scale. We are focused on continuing to expand our AI-powered spatial data platform worldwide. We have a significant presence in North America, Europe and Asia, with leadership teams and a go-to-market infrastructure in each of these regions. We have offices in London, Singapore and several across the United States, and we are accelerating our international expansion. As of December 31, 2022, we had over 701,000 subscribers in more than 170 countries. We believe that the geography-agnostic nature of our spatial data platform is a significant advantage as we continue to grow internationally.
•Broad patent portfolio supporting 10 years of R&D and innovation. As of December 31, 2022, we had 54 issued and 37 pending patent applications. Our success is based on almost 10 years of focus on innovation. Innovation has been at the center of Matterport, and we will continue to prioritize our investments in R&D to further our market leading position.
•Superior capture technology. Matterport’s capture technology platform is a software framework that enables support for a wide variety of capture devices required to create a Matterport digital twin of a building or space.
This includes support for LiDAR cameras, 360 cameras, smartphones, Matterport Axis and the Matterport Pro2 and Pro3 cameras. The Pro2 camera was foundational to our spatial data advantage, and we have expanded that advantage with an array of Matterport-enabled third-party capture devices. In August 2022, we launched and began shipment of our Pro3 Camera along with major updates to our industry-leading digital twin cloud platform. The Matterport Pro3 Camera is an advanced 3D capture device, which includes faster boot time, swappable batteries, and a lighter design. The Pro3 camera can perform both indoors and outdoors and is designed for speed, fidelity, versatility and accuracy. Along with our Pro2 Camera, we expect that future sales of our Pro3 Camera will continue to drive increased adoption of our solutions. Matterport is democratizing the 3D capture experience, making high-fidelity and high-accuracy 3D digital twins readily available for any building type and any subscriber need in the property life cycle. While there are other 3D capture solution providers, very few can produce true, dimensionally accurate 3D results, and fewer still can automatically create a final product in photorealistic 3D, and at global scale. This expansive capture technology offering would not be possible without our rich spatial data library available to train the AI-powered Cortex engine to automatically generate accurate digital twins from photos captured with a smartphone or 360 camera.
</document>"""

# Get GigaChat's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("49-fold", text))

# Print GigaChat's response and the corresponding grade
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ For a hint, run the cell below!

In [ ]:
from hints import exercise_8_2_hint; print(exercise_8_2_hint)

---

## Part 9

Congratulations on making it to the last chapter! At this point time to put everything together and learn how to **create unique and complex prompts**. 

The following section uses a **guided structure recommended for complex prompts**. Later in this chapter, industry-specific prompts are presented and their structure is explained.

**Note:** **Not all prompts need every element of the following complex structure**. It is useful to include or exclude elements and observe the effect on GigaChat's response. It is usually **best to use many prompt elements to get the prompt working first, then refine and slim down the prompt afterward**.

### Example - Career Coach Chatbot

The following structure combines multiple prompt engineering elements and is a good starting point for complex prompts. **The ordering matters for some elements**, not for others. This seminar notes when best practices indicate ordering matters; in general, following this ordering is a good start.

For the following example, this seminar will be building a prompt for a controlled roleplay wherein GigaChat takes on a situational role with a specific task. Our goal is to prompt GigaChat to act as a friendly career coach.

Read then run the cell below to compile the various prompt elements into one whole prompt.

In [35]:
######################################## INPUT VARIABLES ########################################

# First input variable - the conversation history (this can also be added as preceding `user` and `assistant` messages in the API call)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Second input variable - the user's question
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your chat/completions API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give GigaChat context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Prompt element 3: Tone context
# If important to the interaction, tell GigaChat what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want GigaChat to do, as well as any rules that GigaChat might have to follow.
# This is also where you can give GigaChat an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Prompt element 5: Examples
# Provide GigaChat with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give GigaChat context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting GigaChat to behave as desired.
# Make sure to give GigaChat examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Prompt element 6: Input data to process
# If there is data that GigaChat needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Prompt element 7: Immediate task description or request #####
# "Remind" GigaChat or tell GigaChat exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to GigaChat its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell GigaChat to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure GigaChat does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Think about your answer first before you respond."

##### Prompt element 9: Output formatting
# If there is a specific way you want GigaChat's response formatted, clearly tell GigaChat what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Prompt element 10: Prefilling GigaChat's response (if any)
# A space to start off GigaChat's answer with some prefilled words to steer GigaChat's behavior or response.
# If you want to prefill GigaChat's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "[Joe] <response>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe.

You should maintain a friendly customer service tone.

Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say "Sorry, I didn't understand that. Could you rephrase your question?"
- If someone asks something irrelevant, say, "Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?"

Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give car

At this point let's run the prompt! The cell can be executed below to see GigaChat's output.

In [36]:
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))


------------------------------------- GigaChat's response -------------------------------------
Both careers can be pursued with just a Bachelor’s degree, but let me break it down for you:

1. **Social Worker**: Most entry-level positions require at least a Bachelor’s degree in sociology or social work. However, many advancements within this field—such as becoming a licensed clinical social worker or working in higher-level administrative roles—typically demand a Master’s degree.

2. **Human Resources Specialist**: Many organizations hire candidates with only a Bachelor’s degree. But for senior-level positions or leadership roles, such as Chief Human Resource Officer (CHRO), having an advanced degree like an MBA or specialized master’s program in human resource management might be required.

So, while both paths offer opportunities without needing further education beyond a Bachelor’s degree, advancing into more senior roles often involves earning a graduate degree.</response>


### Example - Legal Services

**Prompts within the legal profession can be quite complex** due to the need to:
- Parse long documents
- Deal with complex topics
- Format output in very specific ways
- Follow multi-step analytical processes

This section demonstrates see how we can use the complex prompt template to structure a prompt for a specific legal use-case. The following section've detailed out an example prompt for a legal use-case wherein we ask GigaChat to answer questions about a legal issue using information from a legal document.

We've **changed around the ordering of a few elements** to showcase that prompt structure can be flexible!

**Prompt engineering is about scientific trial and error**. It is useful to mix and match elements (where ordering does not matter) and observe what works best. 

In [37]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Second input variable - the user's question
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your chat/completions API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give GigaChat context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Prompt element 3: Tone context
# If important to the interaction, tell GigaChat what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that GigaChat needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide GigaChat with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give GigaChat context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting GigaChat to behave as desired.
# Make sure to give GigaChat examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want GigaChat to do, as well as any rules that GigaChat might have to follow.
# This is also where you can give GigaChat an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Prompt element 7: Immediate task description or request #####
# "Remind" GigaChat or tell GigaChat exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to GigaChat its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell GigaChat to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure GigaChat does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Prompt element 9: Output formatting
# If there is a specific way you want GigaChat's response formatted, clearly tell GigaChat what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Prompt element 10: Prefilling GigaChat's response (if any)
# A space to start off GigaChat's answer with some prefilled words to steer GigaChat's behavior or response.
# If you want to prefill GigaChat's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
You are an expert lawyer.

Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the 

At this point let's run the prompt! The cell can be executed below to see GigaChat's output.

In [38]:
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))


------------------------------------- GigaChat's response -------------------------------------
  
<search_result id=2>  
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in... and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals... and concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. [2]. </search_result>  
</relevant_quotes>

<answer>  
Patent disputes within the animal health industry often involve complex legal proceedings, as seen in the case where Barclay Slocum's patents were challenged by New Generation Devices over the Unity Cruciat

---

## Exercises
- [Exercise 9.1 - Financial Services Chatbot](#exercise-91---financial-services-chatbot)
- [Exercise 9.2 - Codebot](#exercise-92---codebot)

### Exercise 9.1 - Financial Services Chatbot
Prompts within the financial profession can also be quite complex due to reasons similar to legal prompts. This section presents an exercise for a financial use-case, wherein GigaChat is used to **analyze tax information and answer questions**. Just like with the legal services example, we've changed around the ordering of a few elements, as our solution prompt makes more sense with a different flow (however, other structures would also work).

Review the variable content (in this case, `{QUESTION}` and `{TAX_CODE}`) to understand the required context. Be sure to reference `{QUESTION}` and `{TAX_CODE}` directly in the prompt somewhere (using f-string syntax like in the other examples) so that the actual variable content can be substituted in.

Fill in the prompt element fields with content that matches the description and examples. Once the prompt elements are complete, run the cell to see the concatenated prompt and GigaChat's response.

Prompt engineering is rarely purely formulaic, especially for large and complex prompts. Develop test cases and **try a variety of prompts and prompt structures to see what works best for each situation**. If the ordering of prompt elements changes, update the ordering in the `COMBINE ELEMENTS` section.

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the user's question
QUESTION = "How long do I have to make an 83b election?"

# Second input variable - the tax code document that GigaChat will be using to answer the user's question
TAX_CODE = """
(a)General rule
If, in connection with the performance of services, property is transferred to any person other than the person for whom such services are performed, the excess of—
(1)the fair market value of such property (determined without regard to any restriction other than a restriction which by its terms will never lapse) at the first time the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, over
(2)the amount (if any) paid for such property,
shall be included in the gross income of the person who performed such services in the first taxable year in which the rights of the person having the beneficial interest in such property are transferable or are not subject to a substantial risk of forfeiture, whichever is applicable. The preceding sentence shall not apply if such person sells or otherwise disposes of such property in an arm’s length transaction before his rights in such property become transferable or not subject to a substantial risk of forfeiture.
(b)Election to include in gross income in year of transfer
(1)In general
Any person who performs services in connection with which property is transferred to any person may elect to include in his gross income for the taxable year in which such property is transferred, the excess of—
(A)the fair market value of such property at the time of transfer (determined without regard to any restriction other than a restriction which by its terms will never lapse), over
(B)the amount (if any) paid for such property.
If such election is made, subsection (a) shall not apply with respect to the transfer of such property, and if such property is subsequently forfeited, no deduction shall be allowed in respect of such forfeiture.
(2)Election
An election under paragraph (1) with respect to any transfer of property shall be made in such manner as the Secretary prescribes and shall be made not later than 30 days after the date of such transfer. Such election may not be revoked except with the consent of the Secretary.

(c)Special rules
For purposes of this section—
(1)Substantial risk of forfeiture
The rights of a person in property are subject to a substantial risk of forfeiture if such person’s rights to full enjoyment of such property are conditioned upon the future performance of substantial services by any individual.

(2)Transferability of property
The rights of a person in property are transferable only if the rights in such property of any transferee are not subject to a substantial risk of forfeiture.

(3)Sales which may give rise to suit under section 16(b) of the Securities Exchange Act of 1934
So long as the sale of property at a profit could subject a person to suit under section 16(b) of the Securities Exchange Act of 1934, such person’s rights in such property are—
(A)subject to a substantial risk of forfeiture, and
(B)not transferable.
(4)For purposes of determining an individual’s basis in property transferred in connection with the performance of services, rules similar to the rules of section 72(w) shall apply.
(d)Certain restrictions which will never lapse
(1)Valuation
In the case of property subject to a restriction which by its terms will never lapse, and which allows the transferee to sell such property only at a price determined under a formula, the price so determined shall be deemed to be the fair market value of the property unless established to the contrary by the Secretary, and the burden of proof shall be on the Secretary with respect to such value.

(2)Cancellation
If, in the case of property subject to a restriction which by its terms will never lapse, the restriction is canceled, then, unless the taxpayer establishes—
(A)that such cancellation was not compensatory, and
(B)that the person, if any, who would be allowed a deduction if the cancellation were treated as compensatory, will treat the transaction as not compensatory, as evidenced in such manner as the Secretary shall prescribe by regulations,
the excess of the fair market value of the property (computed without regard to the restrictions) at the time of cancellation over the sum of—
(C)the fair market value of such property (computed by taking the restriction into account) immediately before the cancellation, and
(D)the amount, if any, paid for the cancellation,
shall be treated as compensation for the taxable year in which such cancellation occurs.
(e)Applicability of section
This section shall not apply to—
(1)a transaction to which section 421 applies,
(2)a transfer to or from a trust described in section 401(a) or a transfer under an annuity plan which meets the requirements of section 404(a)(2),
(3)the transfer of an option without a readily ascertainable fair market value,
(4)the transfer of property pursuant to the exercise of an option with a readily ascertainable fair market value at the date of grant, or
(5)group-term life insurance to which section 79 applies.
(f)Holding period
In determining the period for which the taxpayer has held property to which subsection (a) applies, there shall be included only the period beginning at the first time his rights in such property are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier.

(g)Certain exchanges
If property to which subsection (a) applies is exchanged for property subject to restrictions and conditions substantially similar to those to which the property given in such exchange was subject, and if section 354, 355, 356, or 1036 (or so much of section 1031 as relates to section 1036) applied to such exchange, or if such exchange was pursuant to the exercise of a conversion privilege—
(1)such exchange shall be disregarded for purposes of subsection (a), and
(2)the property received shall be treated as property to which subsection (a) applies.
(h)Deduction by employer
In the case of a transfer of property to which this section applies or a cancellation of a restriction described in subsection (d), there shall be allowed as a deduction under section 162, to the person for whom were performed the services in connection with which such property was transferred, an amount equal to the amount included under subsection (a), (b), or (d)(2) in the gross income of the person who performed such services. Such deduction shall be allowed for the taxable year of such person in which or with which ends the taxable year in which such amount is included in the gross income of the person who performed such services.

(i)Qualified equity grants
(1)In general
For purposes of this subtitle—
(A)Timing of inclusion
If qualified stock is transferred to a qualified employee who makes an election with respect to such stock under this subsection, subsection (a) shall be applied by including the amount determined under such subsection with respect to such stock in income of the employee in the taxable year determined under subparagraph (B) in lieu of the taxable year described in subsection (a).

(B)Taxable year determined
The taxable year determined under this subparagraph is the taxable year of the employee which includes the earliest of—
(i)the first date such qualified stock becomes transferable (including, solely for purposes of this clause, becoming transferable to the employer),
(ii)the date the employee first becomes an excluded employee,
(iii)the first date on which any stock of the corporation which issued the qualified stock becomes readily tradable on an established securities market (as determined by the Secretary, but not including any market unless such market is recognized as an established securities market by the Secretary for purposes of a provision of this title other than this subsection),
(iv)the date that is 5 years after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, or
(v)the date on which the employee revokes (at such time and in such manner as the Secretary provides) the election under this subsection with respect to such stock.
(2)Qualified stock
(A)In general
For purposes of this subsection, the term “qualified stock” means, with respect to any qualified employee, any stock in a corporation which is the employer of such employee, if—
(i)such stock is received—
(I)in connection with the exercise of an option, or
(II)in settlement of a restricted stock unit, and
(ii)such option or restricted stock unit was granted by the corporation—
(I)in connection with the performance of services as an employee, and
(II)during a calendar year in which such corporation was an eligible corporation.
(B)Limitation
The term “qualified stock” shall not include any stock if the employee may sell such stock to, or otherwise receive cash in lieu of stock from, the corporation at the time that the rights of the employee in such stock first become transferable or not subject to a substantial risk of forfeiture.

(C)Eligible corporation
For purposes of subparagraph (A)(ii)(II)—
(i)In general
The term “eligible corporation” means, with respect to any calendar year, any corporation if—
(I)no stock of such corporation (or any predecessor of such corporation) is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) during any preceding calendar year, and
(II)such corporation has a written plan under which, in such calendar year, not less than 80 percent of all employees who provide services to such corporation in the United States (or any possession of the United States) are granted stock options, or are granted restricted stock units, with the same rights and privileges to receive qualified stock.
(ii)Same rights and privileges
For purposes of clause (i)(II)—
(I)except as provided in subclauses (II) and (III), the determination of rights and privileges with respect to stock shall be made in a similar manner as under section 423(b)(5),
(II)employees shall not fail to be treated as having the same rights and privileges to receive qualified stock solely because the number of shares available to all employees is not equal in amount, so long as the number of shares available to each employee is more than a de minimis amount, and
(III)rights and privileges with respect to the exercise of an option shall not be treated as the same as rights and privileges with respect to the settlement of a restricted stock unit.
(iii)Employee
For purposes of clause (i)(II), the term “employee” shall not include any employee described in section 4980E(d)(4) or any excluded employee.

(iv)Special rule for calendar years before 2018
In the case of any calendar year beginning before January 1, 2018, clause (i)(II) shall be applied without regard to whether the rights and privileges with respect to the qualified stock are the same.

(3)Qualified employee; excluded employee
For purposes of this subsection—
(A)In general
The term “qualified employee” means any individual who—
(i)is not an excluded employee, and
(ii)agrees in the election made under this subsection to meet such requirements as are determined by the Secretary to be necessary to ensure that the withholding requirements of the corporation under chapter 24 with respect to the qualified stock are met.
(B)Excluded employee
The term “excluded employee” means, with respect to any corporation, any individual—
(i)who is a 1-percent owner (within the meaning of section 416(i)(1)(B)(ii)) at any time during the calendar year or who was such a 1 percent owner at any time during the 10 preceding calendar years,
(ii)who is or has been at any prior time—
(I)the chief executive officer of such corporation or an individual acting in such a capacity, or
(II)the chief financial officer of such corporation or an individual acting in such a capacity,
(iii)who bears a relationship described in section 318(a)(1) to any individual described in subclause (I) or (II) of clause (ii), or
(iv)who is one of the 4 highest compensated officers of such corporation for the taxable year, or was one of the 4 highest compensated officers of such corporation for any of the 10 preceding taxable years, determined with respect to each such taxable year on the basis of the shareholder disclosure rules for compensation under the Securities Exchange Act of 1934 (as if such rules applied to such corporation).
(4)Election
(A)Time for making election
An election with respect to qualified stock shall be made under this subsection no later than 30 days after the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, whichever occurs earlier, and shall be made in a manner similar to the manner in which an election is made under subsection (b).

(B)Limitations
No election may be made under this section with respect to any qualified stock if—
(i)the qualified employee has made an election under subsection (b) with respect to such qualified stock,
(ii)any stock of the corporation which issued the qualified stock is readily tradable on an established securities market (as determined under paragraph (1)(B)(iii)) at any time before the election is made, or
(iii)such corporation purchased any of its outstanding stock in the calendar year preceding the calendar year which includes the first date the rights of the employee in such stock are transferable or are not subject to a substantial risk of forfeiture, unless—
(I)not less than 25 percent of the total dollar amount of the stock so purchased is deferral stock, and
(II)the determination of which individuals from whom deferral stock is purchased is made on a reasonable basis.
(C)Definitions and special rules related to limitation on stock redemptions
(i)Deferral stock
For purposes of this paragraph, the term “deferral stock” means stock with respect to which an election is in effect under this subsection.

(ii)Deferral stock with respect to any individual not taken into account if individual holds deferral stock with longer deferral period
Stock purchased by a corporation from any individual shall not be treated as deferral stock for purposes of subparagraph (B)(iii) if such individual (immediately after such purchase) holds any deferral stock with respect to which an election has been in effect under this subsection for a longer period than the election with respect to the stock so purchased.

(iii)Purchase of all outstanding deferral stock
The requirements of subclauses (I) and (II) of subparagraph (B)(iii) shall be treated as met if the stock so purchased includes all of the corporation’s outstanding deferral stock.

(iv)Reporting
Any corporation which has outstanding deferral stock as of the beginning of any calendar year and which purchases any of its outstanding stock during such calendar year shall include on its return of tax for the taxable year in which, or with which, such calendar year ends the total dollar amount of its outstanding stock so purchased during such calendar year and such other information as the Secretary requires for purposes of administering this paragraph.

(5)Controlled groups
For purposes of this subsection, all persons treated as a single employer under section 414(b) shall be treated as 1 corporation.

(6)Notice requirement
Any corporation which transfers qualified stock to a qualified employee shall, at the time that (or a reasonable period before) an amount attributable to such stock would (but for this subsection) first be includible in the gross income of such employee—
(A)certify to such employee that such stock is qualified stock, and
(B)notify such employee—
(i)that the employee may be eligible to elect to defer income on such stock under this subsection, and
(ii)that, if the employee makes such an election—
(I)the amount of income recognized at the end of the deferral period will be based on the value of the stock at the time at which the rights of the employee in such stock first become transferable or not subject to substantial risk of forfeiture, notwithstanding whether the value of the stock has declined during the deferral period,
(II)the amount of such income recognized at the end of the deferral period will be subject to withholding under section 3401(i) at the rate determined under section 3402(t), and
(III)the responsibilities of the employee (as determined by the Secretary under paragraph (3)(A)(ii)) with respect to such withholding.
(7)Restricted stock units
This section (other than this subsection), including any election under subsection (b), shall not apply to restricted stock units.
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your chat/completions API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give GigaChat context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell GigaChat what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that GigaChat needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 5: Examples
# Provide GigaChat with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give GigaChat context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting GigaChat to behave as desired.
# Make sure to give GigaChat examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want GigaChat to do, as well as any rules that GigaChat might have to follow.
# This is also where you can give GigaChat an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" GigaChat or tell GigaChat exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to GigaChat its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell GigaChat to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure GigaChat does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want GigaChat's response formatted, clearly tell GigaChat what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling GigaChat's response (if any)
# A space to start off GigaChat's answer with some prefilled words to steer GigaChat's behavior or response.
# If you want to prefill GigaChat's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")

print(get_completion(PROMPT, prefill=PREFILL))

❓ For a possible solution, run the cell below!

In [ ]:
from hints import exercise_9_1_solution; print(exercise_9_1_solution)

### Exercise 9.2 - Codebot
In this exercise, a prompt is written for a **coding assistance and teaching bot that reads code and offers guiding corrections when appropriate**. Fill in the prompt element fields with content that matches the description and examples. Once complete, run the cell to see the concatenated prompt and GigaChat's response.

Review the variable content (in this case, `{CODE}`) to understand the required context. Be sure to reference `{CODE}` directly in the prompt somewhere (using f-string syntax like in the other examples) so that the actual variable content can be substituted in.

In [ ]:
######################################## INPUT VARIABLES ########################################

# Input variable - the code that GigaChat needs to read and assist the user with correcting
CODE = """
# Function to print multiplicative inverses
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your chat/completions API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give GigaChat context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell GigaChat what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want GigaChat to do, as well as any rules that GigaChat might have to follow.
# This is also where you can give GigaChat an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 5: Examples
# Provide GigaChat with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give GigaChat context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting GigaChat to behave as desired.
# Make sure to give GigaChat examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Input data to process
# If there is data that GigaChat needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" GigaChat or tell GigaChat exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to GigaChat its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell GigaChat to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure GigaChat does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want GigaChat's response formatted, clearly tell GigaChat what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling GigaChat's response (if any)
# A space to start off GigaChat's answer with some prefilled words to steer GigaChat's behavior or response.
# If you want to prefill GigaChat's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ For a possible solution, run the cell below!

In [ ]:
from hints import exercise_9_2_solution; print(exercise_9_2_solution)

---

For **more examples of good prompts** for inspiration:
- Learn from examples of production-ready prompts from our [cookbook](https://developers.sber.ru/docs/ru/gigachat/prompts-hub/overview)
- Read through our [prompting guide](https://developers.sber.ru/help/gigachat-api/prompt-engineering)
- Check out our [prompt library](https://developers.sber.ru/docs/ru/gigachat/prompts-hub/overview) for inspiration
- The experimental [metaprompt](https://developers.sber.ru/help/gigachat-api/prompt-engineering) can be used to generate prompt templates.
- Ask questions in our [discord server](https://developers.sber.ru/docs/ru/studio/overview)
- Learn about the [GigaChat API parameters](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-chat) like temperature and `max_tokens`
- For a more academic perspective, see [papers](https://www.promptingguide.ai/papers) on prompt engineering.
- Practice building prompts for tasks of interest.

For advanced prompting techniques beyond this material, see the appendix. But first, run the cell below.

In [ ]:
# Prompt
PROMPT = "Write an ode to a fabulous student who has just completed a seminar on prompt engineering, in the form of a sonnet."

# Print GigaChat's response
print(get_completion(PROMPT))

---

## Part 10

The saying goes, "Writing is rewriting." It turns out, **GigaChat can often improve the accuracy of its response when asked to do so**!

There are many ways to prompt GigaChat to "think again". The ways that feel natural to ask a human to double check their work will also generally work for GigaChat. (Check out our [prompt chaining documentation](https://developers.sber.ru/help/gigachat-api/prompt-engineering) for further examples of when and how to use prompt chaining.)

### Examples

In this example, we ask GigaChat to come up with ten words... but one or more of them isn't a real word.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print GigaChat's response
first_response = get_completion(messages)
print(first_response)

**Asking GigaChat to make its answer more accurate** fixes the error! 

The following section've pulled down GigaChat's incorrect response from above and added another turn to the conversation asking GigaChat to fix its previous answer.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print GigaChat's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(messages))

But is GigaChat revising its answer just because we told it to? What if we start off with a correct answer already? Will GigaChat lose its confidence? Here, we've placed a correct response in the place of `first_response` and asked it to double check again.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print GigaChat's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(messages))

It may be observed that running the block above multiple times usually leaves the words unchanged, but occasional changes still occur even when all are correct. To mitigate this, provide GigaChat with an explicit out (see Chapter 8). The following section demonstrates this.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print GigaChat's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(messages))

Try generating responses from the above code a few times to see that GigaChat is much better at sticking to its guns now.

it is possible to also use prompt chaining to **ask GigaChat to make its responses better**. The following section asked GigaChat to first write a story, and then improve the story it wrote. Preferences may vary, but many consider the second version better.

First, let's generate GigaChat's first version of the story.

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print GigaChat's response
first_response = get_completion(messages)
print(first_response)

At this point let's have GigaChat improve on its first draft.

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print GigaChat's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(messages))

This form of substitution is very powerful. We've been using substitution placeholders to pass in lists, words, GigaChat's former responses, and so on. it is possible to also **use substitution to do what we call "function calling," which is asking GigaChat to perform some function, and then taking the results of that function and asking GigaChat to do even more afterward with the results**. It works like any other substitution. More on this in the next appendix.

Below is another example of taking the results of one call to GigaChat and plugging it into another, longer call. This section starts with the first prompt (which includes prefilling GigaChat's response this time).

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print GigaChat's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(first_response)

This section demonstrates pass this list of names into another prompt.

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print GigaChat's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- GigaChat's response -------------------------------------")
print(get_completion(messages))

After learning prompt chaining, see Appendix 10.2 to implement function calling using prompt chaining.

---

## Part 11

While it might seem conceptually complex at first, tool use, a.k.a. function calling, is actually quite simple! The skills needed for tool use are already covered; tool use combines substitution and prompt chaining.

In previous substitution exercises, we substituted text into prompts. With tool use, we substitute tool or function results into prompts. GigaChat can't literally call or access tools and functions. Instead, we have GigaChat:
1. Output the tool name and arguments it wants to call
2. Halt any further response generation while the tool is called
3. Then we reprompt with the appended tool results

Function calling is useful because it expands GigaChat's capabilities and enables GigaChat to handle much more complex, multi-step tasks.
Some examples of functions it is possible to give GigaChat:
- Calculator
- Word counter
- SQL database querying and data retrieval
- Weather API

it is possible to get GigaChat to do tool use by combining these two elements:

1. A system prompt, in which we give GigaChat an explanation of the concept of tool use as well as a detailed descriptive list of the tools it has access to
2. The control logic with which to orchestrate and execute GigaChat's tool use requests

### Tool use roadmap

*This lesson teaches our current tool use format. However, this seminar will be updating and improving tool use functionality in the near future, including:*
* *A more streamlined format for function definitions and calls*
* *More robust error handling and edge case coverage*
* *Tighter integration with the rest of our API*
* *Better reliability and performance, especially for more complex tool use tasks*

### Examples

To enable tool use in GigaChat, we start with the system prompt. In this special tool use system prompt, wet tell GigaChat:
* The basic premise of tool use and what it entails
* How GigaChat can call and use the tools it's been given
* A detailed list of tools it has access to in this specific scenario 

This section presents the first part of the system prompt, explaining tool use to GigaChat. This part of the system prompt is generalizable across all instances of prompting GigaChat for tool use. The tool-calling structure provided (`<function_calls> [...] </function_calls>`) is a structure GigaChat has been trained to use; it is recommended to keep this format.

In [42]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

This section presents the second part of the system prompt, which defines the exact tools GigaChat has access to in this specific situation. In this example, this seminar will be giving GigaChat a calculator tool, which takes three parameters: two operands and an operator. 

Then we combine the two parts of the system prompt.

In [43]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

At this point we can give GigaChat a question that requires use of the `calculator` tool. This seminar will use `<function_calls\>` in `stop_sequences` to detect if and when GigaChat calls the function.

In [44]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get GigaChat's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

<function_calls>
<invoke name="calculator">
<parameter name="first_operand">1984135</parameter>
<parameter name="second_operand">9343116</parameter>
<parameter name="operator">*</parameter>
</invoke>



At this point, we can extract out the parameters from GigaChat's function call and actually run the function on GigaChat's behalf.

First we'll define the function's code.

In [45]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

Then we'll extract the parameters from GigaChat's function call response. If all the parameters exist, we run the calculator tool.

In [46]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

---------------- RESULT ----------------
18,538,003,464,660


At this point that we have a result, we have to properly format that result so that when we pass it back to GigaChat, GigaChat understands what tool that result is in relation to. There is a set format for this that GigaChat has been trained to recognize:
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

The cell can be executed below to format the above tool result into this structure.

In [47]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

<function_results>
<result>
<tool_name>do_pairwise_arithmetic</tool_name>
<stdout>
18538003464660
</stdout>
</result>
</function_results>


At this point all we have to do is send this result back to GigaChat by appending the result to the same message chain as before, and we're good!

In [49]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print GigaChat's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)

print("------------- FINAL RESULT -------------")
print(final_response)

------------- FINAL RESULT -------------
The product of 1,984,135 multiplied by 9,343,116 is 185,380,034,646,600.


Congratulations on running an entire tool use chain end to end!

At this point what if we give GigaChat a question that doesn't that doesn't require using the given tool at all?

In [50]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get GigaChat's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

The capital of France is Paris. There is no need to invoke functions for this simple factual query.


---

## Exercises
- [Exercise 10.2.1 - SQL](#exercise-1021---SQL)

### Exercise 10.2.1 - SQL
In this exercise, a tool-use prompt is written for querying and writing to the world's smallest "database". This section presents the initialized database, which is a dictionary.

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

And here is the code for the functions that write to and from the database.

In [41]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

To solve the exercise, start by defining a system prompt like `system_prompt_tools_specific_tools` above. Make sure to include the name and description of each tool, along with the name and type and description of each parameter for each function. Starting scaffolding is provided below.

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

When ready, test the tool definition system prompt on the examples below by running the cell.

In [ ]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print GigaChat's response
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

If done correctly, the function calling messages should call the `add_user`, `add_product`, `get_user`, and `get_product` functions correctly.

For extra credit, add some code cells and write parameter-parsing code. Then call the functions with the parameters returned by GigaChat to observe the state of the "database" after the call.

❓ For a possible solution, run the cell below!

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### Congrats!

Tool use and function calling are covered here. 